# Building Fast Grammar Fuzzers

## System configuration

In [1]:
!uname -v

Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000


In [2]:
!sw_vers

ProductName:		macOS
ProductVersion:		14.6.1
BuildVersion:		23G93


In [3]:
!system_profiler SPHardwareDataType

Hardware:

    Hardware Overview:

      Model Name: MacBook Pro
      Model Identifier: MacBookPro18,3
      Model Number: MKGQ3X/A
      Chip: Apple M1 Pro
      Total Number of Cores: 10 (8 performance and 2 efficiency)
      Memory: 16 GB
      System Firmware Version: 10151.140.19
      OS Loader Version: 10151.140.19
      Serial Number (system): P9RV0T77R6
      Hardware UUID: 0C70E254-F85C-51D7-A145-AAEB6B290E5C
      Provisioning UDID: 00006000-001031960C06801E
      Activation Lock Status: Disabled



In [4]:
from importlib.metadata import version, PackageNotFoundError
import sys
import subprocess

def ipkg(pkg, repo):
    try:
        version(pkg)
    except PackageNotFoundError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", repo])
    else:
        print(f"{pkg} found")

def ipkgrm(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", pkg])

def ipkginstall(pkg, version):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "%s==%s" % (pkg, version)])

In [5]:
exec_prefix='/'.join(sys.executable.split('/')[0:-1])

In [6]:
ipkginstall('fuzzingbook', '1.01')

In [7]:
ipkg('pickleshare', 'pickleshare')

pickleshare found


In [8]:
from fuzzingbook.Timer import Timer
from fuzzingbook.ExpectError import ExpectTimeout

In [9]:
!rm -rf /tmp/stop.ffg
!rm -rf testers tests
!mkdir -p testers tests

In [10]:
!rm -rf results

**important** We rely on a relatively high recursion limit : 20900 which is only available on MacOSX (Not in Linux).

## Why focus on faster grammar fuzzing?
* Guided fuzzing of uninstrumented code (aka. memory fuzzing).
* Grammar Mining advances

## Building a simple fuzzer

In [11]:
import random
import string
import statistics
import json

In [12]:
def producer(chars, l, n=1):
    return [''.join([random.choice(chars) for i in range(l)]) for i in range(n)]

In [13]:
producer(string.printable, 100)

['@24WAjK$W6}:^<!_YI=;QOjSVdQ}0Qic2}/4q=7\teIr e0{69los7K0fwkvdxE!%]\rB;~u\r}qkqZCKJ5,\x0bxeaC\tQ=p/C/]v}4}xE']

In [14]:
import os, subprocess
from datetime import datetime

## A better tester

In [15]:
import os, subprocess
from datetime import datetime
from resource import getrusage as resource_usage, RUSAGE_CHILDREN
from time import time as timestamp
START_TIME = datetime.now()

In [16]:
class timeit():
    def __init__(self):
        pass
    def __enter__(self):
        self.start_time, self.start_resources = timestamp(), resource_usage(RUSAGE_CHILDREN)
        return self
    def __exit__(self, *args, **kwargs):
        end_time, end_resources = timestamp(), resource_usage(RUSAGE_CHILDREN)
        self._runtime = end_time - self.start_time
        self.sys_runtime = end_resources.ru_stime - self.start_resources.ru_stime 
        self.usr_runtime = end_resources.ru_utime - self.start_resources.ru_utime
        self.runtime = self.sys_runtime + self.usr_runtime

In [17]:
TX = {}

In [18]:
# II
class Tester:
    def __init__(self, name=None, max_num=1000, start_depth=3, limit_depth=5, timeout=3600, iterations=2):
        global TX
        if name is not None:
            self.tname = name
        else:
            self.tname = self.__class__.__name__
        self.tx = TX
        self.max_num, self.start_depth, self.limit_depth, self.timeout, self.iterations = \
            max_num, start_depth, limit_depth, timeout, iterations
        self.tst = {}
        self.tx[self.tname] = self.tst
        self.WARMUP_TIMES = 10
        self.timedout = None
        
    def write_t(self, cmd):
        self.t = "testers/%s-t.sh" % self.tname
        with open(self.t, 'w') as f:
            print('''\
#!/usr/bin/env bash
TIMEFORMAT="%%U %%S";
time %(cmd)s''' % {'cmd':cmd}, file=f)
        !chmod +x {self.t}
        
        
    def init_run(self):
        !rm -rf testers/{self.tname}
        !mkdir -p testers/{self.tname}

    def pre_time(self):
        !rm -rf tests
        !mkdir -p tests
        
    def pre_exec(self, t):
        pass
        
    def exec_program(self, seed, max_depth, t):
        raise NotImplementedError()

    def post_exec(self, t):
        pass
    
    def post_time(self):
        if not self._runtime: return
        if self.file is not None and os.path.exists(self.file):
            lines_cmd = ("cat %s| wc -l" % self.file)
            self.lines = subprocess.getoutput(lines_cmd).strip()
            #unique_cmd = ("cat %s| sort -u| wc -l" % self.file)
            #self.unique_lines = subprocess.getoutput(unique_cmd).strip()
            self.size = os.stat(self.file).st_size
        else:
            self.unique_lines = ''
            self.lines = ''
            self.size = 0
        self.throughput = (self.size/1024/self._runtime, self._runtime)
    
    def timed_exec(self, seed, max_depth, verbose):
        self.pre_time()
        self._runtime = None
        self.timedout = True
        with ExpectTimeout(self.timeout): #, print_traceback=False, mute=True):
            #with timeit() as t:
            t = None
            self.pre_exec(t)
            cmdline = self.exec_program(seed, max_depth, t)
            self.write_t(cmdline)
            !{self.t} 2>./testers/time.out
            self.post_exec(t)
            with open('testers/time.out') as f:
                usr, sys = f.read().strip().split(' ')
            self._runtime = float(usr)+ float(sys)
            self._sys_runtime = float(sys)
            self._usr_runtime = float(usr)
            self.timedout = False
        self.post_time()

    def ofile(self, max_depth, seed):
        fn = 'testers/%s/%d_%d.x' % (self.tname, max_depth, seed)
        return fn
    
    def check_continue(self):
        if os.path.exists('/tmp/stop.ffg'):
            raise Exception('/tmp/stop.ffg -- abort tests')

    def run_test(self, verbose=False):
        def warmup(seed):
            # for warming up, we simply run it a few times before in the
            # same seed as the first, and discount it in computation.
            return [seed]*self.WARMUP_TIMES
        current_time = datetime.now()
        self.init_run()
        # depth is for later when we deal with grammars.
        
        # warmup loop
        for md in [self.start_depth]:
            max_depth = 2**md
            for seed in warmup(0):
                self.file = self.ofile(max_depth, seed)
                self.timed_exec(seed, max_depth, verbose)
                if os.path.exists(self.file): os.remove(self.file)
                
        for md in range(self.start_depth, self.limit_depth):
            max_depth = 2**md
            v = {}
            res = {'detail': v}
            self.tst[max_depth] = res
            seeds = list(range(self.iterations))
            for seed in seeds:
                if self.timedout: break
                self.file = self.ofile(max_depth, seed)
                if verbose: print('depth:', max_depth, 'seed:', seed, 'file:', self.file)
                self.timed_exec(seed, max_depth, verbose)
                if self._runtime:
                    v[seed] = {
                        'runtime':self._runtime,
                        'sys_runtime':self._sys_runtime,
                        'usr_runtime':self._usr_runtime,
                        'size': self.size,
                        #'uniq': self.unique_lines,
                        'lines': self.lines,
                        # in kbytes
                        'throughput': self.size/self._runtime/(1024)}
                if verbose:
                    print(v[seed])
                if os.path.exists(self.file): os.remove(self.file)
                self.check_continue()
            if self.timedout:
                print('Timeout')
                break # we do not expect larger depths to work.
            size = [t['size'] for t in v.values()]
            res['avgsize'] = statistics.mean(size)
                
            sec = [t['runtime'] for t in v.values()]
            res['avgruntime'] = statistics.mean(sec)
            res['stdevruntime'] = statistics.stdev(sec)
                
            tp = [t['throughput'] for t in v.values()]
            res['avgthroughput'] = statistics.mean(tp)
            res['stdevthroughput'] = statistics.stdev(tp)
            print('depth=', max_depth, "size=", res['avgsize'], 'time=', round(res['avgruntime'],3), "stdev(%s)" % str(round(res['stdevruntime'],3)), 'throughput=',res['avgthroughput'], "stdev(%s)" % str(round(res['stdevthroughput'])))
        self.total_test_time = datetime.now() - current_time
        self.dump()
        return self
    
    def dump(self):
        curtime = datetime.now().isoformat()
        name = 'results/%s-tx.json' % (self.tname)
        !mkdir -p results
        with open(name, 'w+') as f:
            print(json.dumps(TX), file=f)
    
    def show(self):
        max_throughput = 0
        best_depth = None
        for depth in self.tst.keys():
            res = self.tst[depth]
            if res.get('avgthroughput',0) > max_throughput:
                max_throughput = res['avgthroughput'] 
                best_depth = depth
        print('Throughput of ', max_throughput, ' kilobytes per second at depth = ', best_depth)
        print("Total time:",str(self.total_test_time))

In [19]:
!cat testers/time.out

cat: testers/time.out: No such file or directory


In [20]:
class RandomTester(Tester):
    def pre_time(self):
        with open('testers/RandomTester/r.py', 'w+') as f:
            print('''
import string,random,sys
random.seed(int(sys.argv[1]))
def producer(chars, l, n=1):
    return [''.join([random.choice(chars) for i in range(l)]) for i in range(n)]
print(producer(string.printable, int(sys.argv[2])))''', file=f)
            
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} testers/RandomTester/r.py {seed} {max_depth} > {fn}"

In [21]:
RandomTester().run_test().show()

depth= 8 size= 14.5 time= 0.035 stdev(0.001) throughput= 0.40450750612745096 stdev(0)
depth= 16 size= 22.5 time= 0.033 stdev(0.0) throughput= 0.6658380681818181 stdev(0)
Throughput of  0.6658380681818181  kilobytes per second at depth =  16
Total time: 0:00:05.164318


In [22]:
TX

{'RandomTester': {8: {'detail': {0: {'runtime': 0.034,
     'sys_runtime': 0.009,
     'usr_runtime': 0.025,
     'size': 14,
     'lines': '1',
     'throughput': 0.4021139705882353},
    1: {'runtime': 0.036000000000000004,
     'sys_runtime': 0.011,
     'usr_runtime': 0.025,
     'size': 15,
     'lines': '1',
     'throughput': 0.40690104166666663}},
   'avgsize': 14.5,
   'avgruntime': 0.035,
   'stdevruntime': 0.0014142135623730963,
   'avgthroughput': 0.40450750612745096,
   'stdevthroughput': 0.0033849704215808056},
  16: {'detail': {0: {'runtime': 0.033,
     'sys_runtime': 0.01,
     'usr_runtime': 0.023,
     'size': 22,
     'lines': '1',
     'throughput': 0.6510416666666666},
    1: {'runtime': 0.033,
     'sys_runtime': 0.01,
     'usr_runtime': 0.023,
     'size': 23,
     'lines': '1',
     'throughput': 0.6806344696969696}},
   'avgsize': 22.5,
   'avgruntime': 0.033,
   'stdevruntime': 0.0,
   'avgthroughput': 0.6658380681818181,
   'stdevthroughput': 0.020925271697

In [23]:
IS_HTML=False

## Grammars

In [24]:
# from fuzzingbook.Parser import make_grammar
# We need a bit of modification make the grammars more varied

In [25]:
def prod_line_grammar(nonterminals, terminals):
    g = {
        '<start>': ['<symbols>'],
        '<symbols>': ['<symbol><symbols>', '<symbol>'],
        '<symbol>': ['<nonterminals>', '<terminals>'],
        '<nonterminals>': ['<lt><alpha><gt>'],
        '<lt>': ['<'],
        '<gt>': ['>'],
        '<alpha>': nonterminals,
        '<terminals>': terminals
    }

    if not nonterminals:
        g['<nonterminals>'] = ['']
        del g['<lt>']
        del g['<alpha>']
        del g['<gt>']

    return g


In [26]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer
from fuzzingbook.Parser import canonical
from fuzzingbook.Grammars import unreachable_nonterminals, RE_NONTERMINAL
import random, string, re

In [27]:
def make_rule(nonterminals, terminals, num_alts):
    prod_grammar = prod_line_grammar(nonterminals, terminals)

    gf = GrammarFuzzer(prod_grammar, min_nonterminals=3, max_nonterminals=5)
    name = "<%s>" % ''.join(random.choices(string.ascii_uppercase, k=3))

    return (name, [gf.fuzz() for _ in range(num_alts)])

In [28]:
make_rule(["A", "B", "C"], ["1", "2", "3"], 3)

('<PJB>', ['122', '2<A><C>2', '323<A>'])

In [29]:
def make_grammar(num_symbols=3, num_alts=3):
    terminals = list(string.ascii_lowercase)
    grammar = {}
    name = None
    for _ in range(num_symbols):
        nonterminals = [k[1:-1] for k in grammar.keys()]
        name, expansions = \
            make_rule(nonterminals, terminals, num_alts)
        grammar[name] = expansions

    grammar['<start>'] = [name]

    # Remove unused parts
    for nonterminal in unreachable_nonterminals(grammar):
        del grammar[nonterminal]
        
    return grammar

In [30]:
canonical(make_grammar())

{'<DAZ>': [['jm'], ['w'], ['ws']],
 '<FZM>': [['<DAZ>', 't', '<DAZ>', 'p'], ['<DAZ>', 'be'], ['<DAZ>', 'a']],
 '<HZS>': [['<FZM>', 'ay'],
  ['<DAZ>', '<DAZ>', '<DAZ>'],
  ['kd', '<DAZ>', '<DAZ>']],
 '<start>': [['<HZS>']]}

### An EXPR grammar

In [31]:
expr_grammar = {
    "<start>": [["<expr>"]],
    "<expr>": [["<term>", "+", "<expr>"], ["<term>", "-", "<expr>"],
               ["<term>"]],
    "<term>": [["<factor>", "*", "<term>"], ["<factor>", "/", "<term>"],
               ["<factor>"]],
    "<factor>": [["+", "<factor>"], ["-", "<factor>"], ["(", "<expr>", ")"],
                 ["<integer>", ".", "<integer>"], ["<integer>"]],
    "<integer>": [["<digit>", "<integer>"], ["<digit>"]],
    "<digit>": [["0"], ["1"], ["2"], ["3"], ["4"], ["5"], ["6"], ["7"], ["8"],
                ["9"]]
}

In [32]:
my_grammar = expr_grammar

Maximum depth is 7; beyond that the size goes > 50G

## Existing tools
* Grammarinator (Python based)
* Gramfuzz (Python based)
* Dharma (Python based)

In [33]:
class Sanitize:
    def __init__(self, g):
        self.g = g
  
    def to_key(self, k):
        s = k.replace('-', '_')
        s = s.replace('[', 'Osq').replace(']','Csq')
        s = s.replace('{','Obr').replace('}','Cbr')
        s = s.replace('import','XimportX')
        s = s.replace('class', 'XclassX')
        s = s.replace('def', 'XdefX')
        return s

    def to_token(self, t):
        return t
    
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        # these should not matter for performance comparisons,
        # and makes my life simpler
        esc = {'\r': '\r', '\n': '\n',
             '\\': '\\',
             '"':'"',
             "'":"'"}
        for i in t:
            if i in esc:
                my_tokens.append(esc[i])
            else:
                my_tokens.append(i)
        return my_tokens
            
        return list(t)

    def to_rule(self, rule, grammar):
        tokens = [k for t in rule for k in self.split_tokens(t, grammar)]
        return [self.to_token(t) if t not in grammar else self.to_key(t)
                for t in tokens]

    def translate(self):
        new_grammar = {}
        for k in self.g:
            rules = self.g[k]
            new_grammar[self.to_key(k)] = [self.to_rule(rule, self.g) for rule in rules]
        return new_grammar

In [34]:
class AntlrG(Sanitize):
    def to_key(self, k):
        return super().to_key(k)[1:-1]

    def esc_token(self, t):
        # these are multi-char tokens
        t = t.replace('\\','\\\\')
        t = t.replace("'","\\\'")
        t = t.replace('\n','\\n')
        t = t.replace('\r','\\r')
        t = t.replace('\t','\\t')
        return t

    def rule_to_s(self, rule, grammar):
        return ' '.join(["'%s'" % self.esc_token(t)
                         if t not in grammar else self.to_key(t)
                         for t in rule])

    def translate(self):
        lines = ['grammar Grammar;']
        for k in self.g:
            rules = self.g[k]
            v = '\n    |'.join([self.rule_to_s(rule, self.g)
                                for rule in rules])
            lines.append('''\
%s
    : %s
    ;''' % (self.to_key(k), v))
        return '\n'.join(lines)

In [35]:
g4 = AntlrG(my_grammar).translate()

In [36]:
with open('testers/grammar.g4', 'w+') as f:
    print(g4, file=f)

### The Fuzzing book approach

In [37]:
def pp_grammar(grammar): return json.dumps(grammar, indent=2, sort_keys=False)

In [38]:
class Trans(Sanitize):
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        esc = {'\r': '\\\r', '\n': '\\\n',
             '\\': '\\\\',
             '"':'\\\"',
             "'":"'"}
        for i in t:
            if i in esc:
                my_tokens.append(esc[i])
            else:
                my_tokens.append(i)
        return my_tokens
            
        return list(t)

    def to_rule(self, rule, grammar):
        tokens = [k for t in rule for k in self.split_tokens(t, grammar)]
        return [self.to_token(t) if t not in grammar else self.to_key(t)
                for t in tokens]

    def translate(self):
        new_grammar = {}
        for k in self.g:
            rules = self.g[k]
            new_grammar[self.to_key(k)] = [self.to_rule(rule, self.g) for rule in rules]
        return new_grammar

In [39]:
s_grammar = Sanitize(my_grammar).translate()

In [40]:
s_grammar

{'<start>': [['<expr>']],
 '<expr>': [['<term>', '+', '<expr>'], ['<term>', '-', '<expr>'], ['<term>']],
 '<term>': [['<factor>', '*', '<term>'],
  ['<factor>', '/', '<term>'],
  ['<factor>']],
 '<factor>': [['+', '<factor>'],
  ['-', '<factor>'],
  ['(', '<expr>', ')'],
  ['<integer>', '.', '<integer>'],
  ['<integer>']],
 '<integer>': [['<digit>', '<integer>'], ['<digit>']],
 '<digit>': [['0'],
  ['1'],
  ['2'],
  ['3'],
  ['4'],
  ['5'],
  ['6'],
  ['7'],
  ['8'],
  ['9']]}

In [41]:
with open('testers/fuzzingbook_gfuzzer.py', 'w+') as f:
    print('''grammar = ''', pp_grammar(s_grammar), file=f)
    print("""
result = ''
from fuzzingbook.GrammarFuzzer import GrammarFuzzer
def canonical(grammar):
    new_g = {}
    for k in grammar:
        new_g[k] = []
        for rule in grammar[k]:
            new_g[k].append(''.join(rule))
    return new_g
import random
def main(args):
    random.seed(int(sys.argv[1]))
    global result
    max_num = int(args[2])
    max_depth = int(args[3])
    fuzzer = GrammarFuzzer(canonical(grammar), max_nonterminals=max_depth)
    global result
    for i in range(max_num):
        result = fuzzer.fuzz()
        print(result)
        result = ''
import sys
main(sys.argv)""", file=f)

In [42]:
!cat testers/fuzzingbook_gfuzzer.py

grammar =  {
  "<start>": [
    [
      "<expr>"
    ]
  ],
  "<expr>": [
    [
      "<term>",
      "+",
      "<expr>"
    ],
    [
      "<term>",
      "-",
      "<expr>"
    ],
    [
      "<term>"
    ]
  ],
  "<term>": [
    [
      "<factor>",
      "*",
      "<term>"
    ],
    [
      "<factor>",
      "/",
      "<term>"
    ],
    [
      "<factor>"
    ]
  ],
  "<factor>": [
    [
      "+",
      "<factor>"
    ],
    [
      "-",
      "<factor>"
    ],
    [
      "(",
      "<expr>",
      ")"
    ],
    [
      "<integer>",
      ".",
      "<integer>"
    ],
    [
      "<integer>"
    ]
  ],
  "<integer>": [
    [
      "<digit>",
      "<integer>"
    ],
    [
      "<digit>"
    ]
  ],
  "<digit>": [
    [
      "0"
    ],
    [
      "1"
    ],
    [
      "2"
    ],
    [
      "3"
    ],
    [
      "4"
    ],
    [
      "5"
    ],
    [
      "6"
    ],
    [
      "7"
    ],
    [
      "8"
    ],
    [
      "9"
    ]
  ]
}

result = ''
from fuzzingbook.

In [43]:
!{sys.executable} testers/fuzzingbook_gfuzzer.py 0 10 10

+-+-+(+8)/-+-435+005.1-6/86
1*4.9*7*(8-1)*-1*1-0/(9-3)+4.4*(3+6)
(7/3+9)/90-0+4*5+0-2
69.8-++8*25.9305
82
8/3/-8-+(4.4/7-1+5)*+-4574.50*18*--4
6.5*--(44/+7.4+2.0*36.47-((1*(5/9*5+7)+6*2-2+0-4)))
---(9*3--6.74)/-++1*9.0*7*0
-2/-6.3/(5)-(3)/(9)+6*0+3-1
+--(-4*2-0-7)*(37.7)/++2.15*1


In [44]:
class FuzzingbookTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} testers/fuzzingbook_gfuzzer.py {seed} {self.max_num} {max_depth} > {fn}"

In [45]:
# The fuzzing book fuzzer does not improve with more depth.
FuzzingbookTester(limit_depth=5).run_test().show()

depth= 8 size= 19665 time= 2.13 stdev(0.013) throughput= 9.014290071546165 stdev(0)
depth= 16 size= 47111 time= 10.063 stdev(0.461) throughput= 4.574883775538085 stdev(0)
Throughput of  9.014290071546165  kilobytes per second at depth =  8
Total time: 0:00:54.544298


## Building a grammar fuzzer

In [46]:
import sys
assert sys.platform == 'darwin'
sys.setrecursionlimit(20900) # for OSX only

In [47]:
class Fuzzer:
    def __init__(self, grammar):
        self.grammar = grammar
    
    def fuzz(self, key='<start>', max_num=None, max_depth=None):
        raise NotImplemented()

In [48]:
class NFuzzer(Fuzzer):
    def gen_key(self, key):
        return (self.gen_rule(random.choice(self.grammar[key]))
                if key in self.grammar else key)

    def gen_rule(self, rule):
        return ''.join(self.gen_key(token) for token in rule)

    def fuzz(self, key='<start>', max_depth=None):
        return self.gen_key(key)

In [49]:
my_fuzzer = NFuzzer(s_grammar)
try:
    for i in range(100):
        print(repr(my_fuzzer.fuzz()))
except:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(exc_type, exc_value)

<class 'RecursionError'> maximum recursion depth exceeded while calling a Python object


In [50]:
import inspect
import json

In [51]:
def get_opts(args, log=False):
    seed = int(args[1])
    max_num = int(args[2])
    max_depth = int(args[3])
    random.seed(seed)
    sys.setrecursionlimit(20900)
    if log:
        print("seed=%d, num=%d, depth=%d" % (seed, max_num, max_depth), file=sys.stderr)
    return max_num, max_depth

In [52]:
def extract_class_definition(cls, log=False):
    eldest = [c for c in cls.mro()
                if c.__name__ == cls.__name__ and
                   cls.__name__ not in {i.__name__ for i in c.__bases__}]
    n_parents = sum([[j.__name__ for j in i.__bases__] for i in eldest], [])
    s_parents = '(%s)' % ', '.join(set(n_parents)) if n_parents else ''
    buf = ["class %s%s:" % (cls.__name__, s_parents)]
    seen = set()
    i = 0
    for curcls in cls.mro():
        i += 1
        if log: print('Parent: %d' % i, curcls.__name__)
        if curcls.__name__ != cls.__name__: continue
        for fn_name in dir(curcls):
            if log: print('\t:', fn_name)
            if fn_name in seen: continue
            if fn_name == '__new__':
                continue
            fn = curcls.__dict__.get(fn_name)
            if fn is None:
                continue
            if ('function' in str(type(fn))):
                seen.add(fn_name)
                buf.append(inspect.getsource(fn))
    return '\n'.join(buf)

In [53]:
def write_file(file_name, grammar, classes, fns=[get_opts], fuzzer=None):
    with open(file_name, 'w+') as f:
        print('''grammar = ''', pp_grammar(grammar), file=f)
        for cls in classes:
            print(extract_class_definition(cls), file=f)
        for fn in fns:
            print(inspect.getsource(fn), file=f)
        print("""
import itertools
import sys
import random
def main(args):
    max_num, max_depth = get_opts(args)
    my_fuzzer = %s(grammar)
    for i in range(max_num):
        print(my_fuzzer.fuzz(key='<start>', max_depth=max_depth))
try:
    main(sys.argv)
    sys.exit(0)
except RecursionError as e:
    print(e, file=sys.stderr)
    sys.exit(2)
""" % fuzzer.__name__, file=f)

In [54]:
write_file('testers/grammar_producer_naive.py', s_grammar, [Fuzzer, NFuzzer], fuzzer=NFuzzer)

In [55]:
!cat testers/grammar_producer_naive.py

grammar =  {
  "<start>": [
    [
      "<expr>"
    ]
  ],
  "<expr>": [
    [
      "<term>",
      "+",
      "<expr>"
    ],
    [
      "<term>",
      "-",
      "<expr>"
    ],
    [
      "<term>"
    ]
  ],
  "<term>": [
    [
      "<factor>",
      "*",
      "<term>"
    ],
    [
      "<factor>",
      "/",
      "<term>"
    ],
    [
      "<factor>"
    ]
  ],
  "<factor>": [
    [
      "+",
      "<factor>"
    ],
    [
      "-",
      "<factor>"
    ],
    [
      "(",
      "<expr>",
      ")"
    ],
    [
      "<integer>",
      ".",
      "<integer>"
    ],
    [
      "<integer>"
    ]
  ],
  "<integer>": [
    [
      "<digit>",
      "<integer>"
    ],
    [
      "<digit>"
    ]
  ],
  "<digit>": [
    [
      "0"
    ],
    [
      "1"
    ],
    [
      "2"
    ],
    [
      "3"
    ],
    [
      "4"
    ],
    [
      "5"
    ],
    [
      "6"
    ],
    [
      "7"
    ],
    [
      "8"
    ],
    [
      "9"
    ]
  ]
}
class Fuzzer(object):
    def 

In [56]:
# the seed and max_num is chosen to avoid recusion error.
!time {sys.executable} testers/grammar_producer_naive.py 1 96 0 > testers/fuzz.out

maximum recursion depth exceeded
/Users/rahul.gopinath/Research/f1/F1/py/bin/python3.12  1 96 0 >   0.04s user 0.01s system 96% cpu 0.050 total


## Setting expansion limits

We can compute the least cost paths to take and use only those paths after a given depth is exceeded.

In [57]:
import sys
import functools

In [58]:
class LimitFuzzer(Fuzzer):
    def symbol_cost(self, grammar, symbol, seen):
        if symbol in self.key_cost: return self.key_cost[symbol]
        if symbol in seen:
            self.key_cost[symbol] = float('inf')
            return float('inf')
        v = min((self.expansion_cost(grammar, rule, seen | {symbol})
                    for rule in grammar.get(symbol, [])), default=0)
        self.key_cost[symbol] = v
        return v

    def expansion_cost(self, grammar, tokens, seen):
        return max((self.symbol_cost(grammar, token, seen)
                    for token in tokens if token in grammar), default=0) + 1

In [59]:
class LimitFuzzer(LimitFuzzer):
    def gen_key(self, key, depth, max_depth):
        if key not in self.grammar: return key
        if depth > max_depth:
            clst = sorted([(self.cost[key][str(rule)], rule) for rule in self.grammar[key]])
            rules = [r for c,r in clst if c == clst[0][0]]
        else:
            rules = self.grammar[key]
        return self.gen_rule(random.choice(rules), depth+1, max_depth)

    def gen_rule(self, rule, depth, max_depth):
        return ''.join(self.gen_key(token, depth, max_depth) for token in rule)

    def fuzz(self, key='<start>', max_depth=10):
        return self.gen_key(key=key, depth=0, max_depth=max_depth)

In [60]:
class LimitFuzzer(LimitFuzzer):
    def __init__(self, grammar):
        super().__init__(grammar)
        self.key_cost = {}
        self.cost = self.compute_cost(grammar)
 
    def compute_cost(self, grammar):
        cost = {}
        for k in grammar:
            cost[k] = {}
            for rule in grammar[k]:
                cost[k][str(rule)] = self.expansion_cost(grammar, rule, set())  
        return cost

In [61]:
my_fuzzer = LimitFuzzer(s_grammar)

In [62]:
my_fuzzer.fuzz()

'-(+2*-8*(5)/-6/8.3)/((+7/6*2.5+3/7-1+5)+326.560-+5/9.3*1/2.0--1.6*6+2.9+5)++(5.134+(0))*2.89294+++(+5*1.6+1*1.0+5.2-9)/+-+5.0'

In [63]:
write_file('testers/grammar_producer_limit.py', s_grammar, [Fuzzer, LimitFuzzer], fuzzer=LimitFuzzer)

In [64]:
!cat testers/grammar_producer_limit.py

grammar =  {
  "<start>": [
    [
      "<expr>"
    ]
  ],
  "<expr>": [
    [
      "<term>",
      "+",
      "<expr>"
    ],
    [
      "<term>",
      "-",
      "<expr>"
    ],
    [
      "<term>"
    ]
  ],
  "<term>": [
    [
      "<factor>",
      "*",
      "<term>"
    ],
    [
      "<factor>",
      "/",
      "<term>"
    ],
    [
      "<factor>"
    ]
  ],
  "<factor>": [
    [
      "+",
      "<factor>"
    ],
    [
      "-",
      "<factor>"
    ],
    [
      "(",
      "<expr>",
      ")"
    ],
    [
      "<integer>",
      ".",
      "<integer>"
    ],
    [
      "<integer>"
    ]
  ],
  "<integer>": [
    [
      "<digit>",
      "<integer>"
    ],
    [
      "<digit>"
    ]
  ],
  "<digit>": [
    [
      "0"
    ],
    [
      "1"
    ],
    [
      "2"
    ],
    [
      "3"
    ],
    [
      "4"
    ],
    [
      "5"
    ],
    [
      "6"
    ],
    [
      "7"
    ],
    [
      "8"
    ],
    [
      "9"
    ]
  ]
}
class Fuzzer(object):
    def 

In [65]:
class PyLimitTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} ./testers/grammar_producer_limit.py {seed} {self.max_num} {max_depth} > {fn}"

In [66]:
!{sys.executable} testers/grammar_producer_limit.py 0 10 10

(7.9/418*417)*+(87/(0+1)---1+1/8.8/1.5+3.0*1.2)-8*39/5.573/+(2+2-1)*-65+313*++9++-+(8-3)/(3/5*2)/9.5*(5.8+5)/(6.0)+(-9.0*2.1/8+7-6.8-6)*+-+3/42*1.4*2/4
-((7*8.3)/0/-(5.0)*-5/1/4)*45*17.5/70.5*--1+--+18.7*654651/+3+3.3175/-+2+5.7/-05
-(7.1124*2.78*98/+4.7)+028-((0.2/0.7-2.6-0.7)/++1*3+++1.2/(7)/5/7-3*1.6*6-0.6*5+1)/((5-3)*+5)*+277
-+-((1-2.5)/-4-+6/6*9.4-7/6)/7.57*-+(7+2.7*7-4.7)*31.8
784.10-7.44/-048+264*-864-(--6)/2/(3.7-7.2)-+07/+10/++4/-3
((78.2-6.7*7/0+5+9+7.8)*9-(-0.0/4.2*9.8)/595.8*(5-3.4)/4/0.8/7-+7.1/+-6*6.0*3*5.1)/(-(4.0-3+0))-(3+3)/-+5+9.285/+-227.335+-987.4
-8.1+3*(-+(4.2)*+(7.0)/+5*6*6-(0.4+6)/+1*3/1++9.4-0.1-3-4.6)/(0/0)*67.00/3/6-+3*2-+954/(2.2/2-9.1*8.1-6)+((6.0)/5+0.8*7-2+7.3)/64.6*1/+4.6/0.5/0.7-++5*+-0*+2.6/6
---882.9*+444*2/08539/5788.0*+9.5*1/(5)*6
95679.4/2.73
+++(58.7*+0.1*7.2/6+2*9*6-5*3+9.6+2)/++(++2*6.9)*(-(2.9)*(7.3+9.4))+(3*+6/+2)-+6.72/(-8/(8.3)*4/8-6.5*9*6-1.8/3)*-+79*(4/8.6)


In [67]:
PyLimitTester().run_test().show()

depth= 8 size= 51554.5 time= 0.169 stdev(0.0) throughput= 297.9064580251479 stdev(6)
depth= 16 size= 240401.5 time= 0.559 stdev(0.007) throughput= 420.00049857019866 stdev(4)
Throughput of  420.00049857019866  kilobytes per second at depth =  16
Total time: 0:00:11.701443


## Using precomputed string pools

**idea**: We can precompute the closing parts.

In [68]:
class PooledFuzzer(LimitFuzzer):
    def compute_cost(self, grammar, cost={}):
        return {k:sorted([(self.expansion_cost(grammar, rule, set()), rule)
                          for rule in grammar[k]])
                for k in self.grammar}

In [69]:
class PooledFuzzer(PooledFuzzer):
    def cheap_grammar(self):
        new_grammar = {}
        for k in self.cost:
            crules = self.cost[k]
            min_cost = crules[0][0]
            new_grammar[k] = [r for c,r in crules if c == min_cost]
            assert len(new_grammar[k]) > 0
        return new_grammar

In [70]:
PooledFuzzer(s_grammar).cheap_grammar()

{'<start>': [['<expr>']],
 '<expr>': [['<term>']],
 '<term>': [['<factor>']],
 '<factor>': [['<integer>'], ['<integer>', '.', '<integer>']],
 '<integer>': [['<digit>']],
 '<digit>': [['0'],
  ['1'],
  ['2'],
  ['3'],
  ['4'],
  ['5'],
  ['6'],
  ['7'],
  ['8'],
  ['9']]}

In [71]:
import itertools
import random

In [72]:
class PooledFuzzer(PooledFuzzer):
    def get_strings_for_key(self, grammar, key='<start>'):
        if key not in grammar: return [key]
        v = sum([self.get_strings_for_rule(grammar, rule)
                 for rule in grammar[key]], [])
        return random.sample(v, min(self.MAX_SAMPLE, len(v)))

    def get_strings_for_rule(self, grammar, rule):
        my_strings_list = [self.get_strings_for_key(grammar, key) for key in rule]
        v = [''.join(l) for l in itertools.product(*my_strings_list)]
        return random.sample(v, min(self.MAX_SAMPLE, len(v)))

    def completion_strings(self):
        # we are being choosy
        return {k:self.get_strings_for_key(self.c_grammar, k)
                for k in self.c_grammar}

In [73]:
pf = PooledFuzzer(s_grammar)

In [74]:
pf.c_grammar = pf.cheap_grammar()

In [75]:
pf.MAX_SAMPLE = 255
strings = pf.completion_strings()

In [76]:
for k in strings:
    print(k, strings[k])

<start> ['3.7', '0.2', '9.2', '3.9', '5.7', '8.8', '8.3', '1.9', '1.1', '2.9', '1.5', '1.7', '6.4', '2.4', '2.0', '7', '2.3', '5.9', '2.5', '5', '4.6', '8.9', '1.6', '2.8', '7.5', '3.8', '4.3', '9.4', '3.5', '0.1', '0.5', '1.3', '0.9', '9.8', '6.8', '1.8', '9.3', '5.8', '5.0', '3.0', '8', '8.7', '6.3', '4.9', '0.4', '7.3', '4.7', '4.5', '3.6', '9.9', '9.0', '9.6', '3.1', '2.1', '6', '4.0', '7.7', '8.4', '2.2', '7.1', '9.5', '5.3', '3.2', '6.2', '3', '0', '8.0', '7.0', '0.8', '1.2', '9.7', '4.1', '0.6', '8.5', '2.6', '6.7', '5.2', '4.8', '3.3', '6.5', '1.4', '5.5', '8.6', '2.7', '1.0', '2', '6.9', '0.7', '0.0', '4.2', '9', '1', '3.4', '0.3', '6.6', '4', '6.0', '7.6', '8.1', '9.1', '7.4', '7.9', '5.4', '7.8', '8.2', '5.6', '4.4', '5.1', '7.2', '6.1']
<expr> ['9.4', '0.5', '6.0', '7.1', '0.2', '9.2', '0.0', '4.7', '7.7', '9.7', '1.1', '6.3', '9.1', '1.6', '2.9', '7.3', '3.2', '0', '7.9', '3.6', '8.7', '2.6', '9.5', '2.2', '3.5', '1.3', '1.5', '7.6', '6.7', '5.1', '8.5', '4.6', '5', '2.8',

In [77]:
class PooledFuzzer(PooledFuzzer):
    def __init__(self, grammar):
        super().__init__(grammar)
        self.c_grammar = self.cheap_grammar()
        self.MAX_SAMPLE = 255
        self.pool_of_strings = self.completion_strings()
        # reorder our grammar rules by cost.
        for k in self.grammar:
            self.grammar[k] = [r for (i,r) in self.cost[k]]
        self.ordered_grammar = True
        
    def gen_key(self, key, depth, max_depth):
        if key not in self.grammar: return key
        if depth > max_depth:
            return random.choice(self.pool_of_strings[key])
        return self.gen_rule(random.choice(self.grammar[key]), depth+1, max_depth)

In [78]:
my_fuzzer = PooledFuzzer(s_grammar)
for i in range(10):
    print(repr(my_fuzzer.fuzz()))

'(3/++21.08*+(6.6)+47*-03/(8.5)*0/2.2+++(7.4)*(5.6+4.7)/3.9/6.2+(3.0)/-7.1-3/7.3)/(-++3--(5-6.0)/+0/1/5.7-93.46*-8.3/6.6+(6)*9.6*1.4+1.0)*6.65/5'
'66042.11'
'--6+1*+(-9/+1.7/2.6+8.9)*(-(8.1)/+0++7.3*6.2/1.1)'
'-+7'
'++8*(-2.7-(0/2.6)/-4.8*-8.9*2.5/5.5-(7.9)*+7.8*5.3)*((9.0-0.2+8.8))/--2.51/-+41-77/4.8'
'136542/7/(42/49*-9.3*2.2/7.7+17.7)/-4324-(-092)/1*2'
'4*-+2'
'+(97.9*334*-+6.6)+5*5*37062/53*(5.1*4.4)+1.364*02/-(2/6.1+8.9+5.6)/(8.7/0-2.7)*47-++7.2/+---4.0/745'
'128.1/867/503.9/-+(9.3*8.9+8.4)'
'(+564*-(8/5.2-3.9+0.1)*-8*(7-0.6))*8*-(8*-1.4/1.1/1-4*4.6-7.7+5.6)*9130*4.5*+--2.2/+(0.6)*+6.8'


In [79]:
write_file('testers/grammar_producer_pool.py', s_grammar, [Fuzzer, LimitFuzzer, PooledFuzzer], fuzzer=PooledFuzzer)

In [80]:
!cat testers/grammar_producer_pool.py

grammar =  {
  "<start>": [
    [
      "<expr>"
    ]
  ],
  "<expr>": [
    [
      "<term>"
    ],
    [
      "<term>",
      "+",
      "<expr>"
    ],
    [
      "<term>",
      "-",
      "<expr>"
    ]
  ],
  "<term>": [
    [
      "<factor>"
    ],
    [
      "<factor>",
      "*",
      "<term>"
    ],
    [
      "<factor>",
      "/",
      "<term>"
    ]
  ],
  "<factor>": [
    [
      "<integer>"
    ],
    [
      "<integer>",
      ".",
      "<integer>"
    ],
    [
      "+",
      "<factor>"
    ],
    [
      "-",
      "<factor>"
    ],
    [
      "(",
      "<expr>",
      ")"
    ]
  ],
  "<integer>": [
    [
      "<digit>"
    ],
    [
      "<digit>",
      "<integer>"
    ]
  ],
  "<digit>": [
    [
      "0"
    ],
    [
      "1"
    ],
    [
      "2"
    ],
    [
      "3"
    ],
    [
      "4"
    ],
    [
      "5"
    ],
    [
      "6"
    ],
    [
      "7"
    ],
    [
      "8"
    ],
    [
      "9"
    ]
  ]
}
class Fuzzer(object):
    def 

In [81]:
class PyPooledTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} testers/grammar_producer_pool.py {seed} {self.max_num} {max_depth} > {fn}"

In [82]:
PyPooledTester().run_test().show()

depth= 8 size= 57931.5 time= 0.075 stdev(0.003) throughput= 754.9933946083881 stdev(36)
depth= 16 size= 249138.5 time= 0.209 stdev(0.015) throughput= 1163.5161670390303 stdev(62)
Throughput of  1163.5161670390303  kilobytes per second at depth =  16
Total time: 0:00:09.796357


Can we do better?

## Compile the Grammar

In [83]:
class PyTrans(Sanitize):
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        for i in t:
            my_tokens.append(i)
        return my_tokens

In [84]:
pyc_grammar = PyTrans(my_grammar).translate()

In [85]:
pyc_grammar

{'<start>': [['<expr>']],
 '<expr>': [['<term>', '+', '<expr>'], ['<term>', '-', '<expr>'], ['<term>']],
 '<term>': [['<factor>', '*', '<term>'],
  ['<factor>', '/', '<term>'],
  ['<factor>']],
 '<factor>': [['+', '<factor>'],
  ['-', '<factor>'],
  ['(', '<expr>', ')'],
  ['<integer>', '.', '<integer>'],
  ['<integer>']],
 '<integer>': [['<digit>', '<integer>'], ['<digit>']],
 '<digit>': [['0'],
  ['1'],
  ['2'],
  ['3'],
  ['4'],
  ['5'],
  ['6'],
  ['7'],
  ['8'],
  ['9']]}

### Compile to Python

In [86]:
# not clear what is the fastest: + or ''.join
# https://stackoverflow.com/questions/1316887/what-is-the-most-efficient-string-concatenation-method-in-python
class PyCompiledFuzzer(PooledFuzzer):
    def add_indent(self, string, indent):
        return '\n'.join([indent + i for i in string.split('\n')])

    # used for escaping inside strings
    def esc(self, t):
        t = t.replace('\\', '\\\\')
        t = t.replace('\n', '\\n')
        t = t.replace('\r', '\\r')
        t = t.replace('\t', '\\t')
        t = t.replace('\b', '\\b')
        t = t.replace('\v', '\\v')
        t = t.replace('"', '\\"')
        return t
    
    def esc_char(self, t):
        assert len(t) == 1
        t = t.replace('\\', '\\\\')
        t = t.replace('\n', '\\n')
        t = t.replace('\r', '\\r')
        t = t.replace('\t', '\\t')
        t = t.replace('\b', '\\b')
        t = t.replace('\v', '\\v')
        t = t.replace("'", "\\'")
        return t

    def k_to_s(self, k): return k[1:-1].replace('-', '_')

    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                res.append('''\
gen_%s(next_depth, max_depth)''' % self.k_to_s(token))
            else:
                res.append('''\
result.append("%s")''' % self.esc(token))
        return '\n'.join(res)

    def string_pool_defs(self):
        result =[]
        for k in self.pool_of_strings:
            result.append('''\
pool_of_%(key)s = %(values)s''' % {
                'key':self.k_to_s(k),
                'values': self.pool_of_strings[k]})
        result.append('''
result = []''')
        return '\n'.join(result)

    def gen_main_src(self):
        result = []
        result.append('''
import random
import sys
def main(args):
    global result
    max_num, max_depth = get_opts(args)
    for i in range(max_num):
        gen_start(0, max_depth)
        print(''.join(result))
        result = []
 
main(sys.argv)''')
        return '\n'.join(result)

    def gen_alt_src(self, key):
        rules = self.grammar[key]
        result = []
        result.append('''
def gen_%(name)s(depth, max_depth):
    next_depth = depth + 1
    if depth > max_depth:
        result.append(random.choice(pool_of_%(name)s))
        return
    val = random.randrange(%(nrules)s)''' % {
            'name':self.k_to_s(key),
            'nrules':len(rules)})
        for i, rule in enumerate(rules):
            result.append('''\
    if val == %d:
%s
        return''' % (i, self.add_indent(self.gen_rule_src(rule, key, i),'        ')))
        return '\n'.join(result)

    def gen_fuzz_src(self):
        result = []
        result.append(self.string_pool_defs())
        for key in self.grammar:
            result.append(self.gen_alt_src(key))
        return '\n'.join(result)

    def fuzz_src(self, key='<start>'):
        result = [self.gen_fuzz_src(),
                  self.gen_main_src()]
        return ''.join(result)

In [87]:
with open('testers/grammar_producer_pycompiled.py', 'w+') as f:
    for fn in [get_opts]:
        print(inspect.getsource(fn), file=f)
    result = PyCompiledFuzzer(pyc_grammar).fuzz_src()
    print(result, file=f)

In [88]:
!cat testers/grammar_producer_pycompiled.py

def get_opts(args, log=False):
    seed = int(args[1])
    max_num = int(args[2])
    max_depth = int(args[3])
    random.seed(seed)
    sys.setrecursionlimit(20900)
    if log:
        print("seed=%d, num=%d, depth=%d" % (seed, max_num, max_depth), file=sys.stderr)
    return max_num, max_depth

pool_of_start = ['8.2', '1.1', '0.5', '6.9', '4.7', '2.5', '5', '1.8', '7.7', '8.5', '9.9', '3.3', '0.4', '3.5', '9.8', '0', '0.2', '9.5', '0.3', '6.6', '2.6', '4.6', '8.7', '8.4', '1.7', '4.0', '6.0', '5.0', '4.5', '0.9', '3.7', '2.2', '8.3', '7.3', '3.8', '6.8', '5.6', '5.4', '0.6', '6.2', '0.0', '1.3', '8.6', '6.3', '7', '7.4', '5.8', '4.3', '5.9', '2.4', '6.7', '1.9', '3.0', '2.0', '3.9', '9.1', '4.2', '1.6', '5.7', '9.2', '5.2', '1.5', '7.9', '9.7', '2.8', '6.4', '4.9', '8.8', '8', '7.1', '5.1', '4.8', '0.1', '7.8', '9', '0.7', '8.1', '7.0', '3', '7.2', '4.1', '4.4', '9.0', '1', '0.8', '9.6', '1.2', '3.6', '3.2', '6.1', '1.4', '5.5', '5.3', '7.6', '2.1', '3.1', '8.9', '2', '2.3', '1.0', '

In [89]:
!{sys.executable} testers/grammar_producer_pycompiled.py 0 10 10

+(+-+(6)*20.82/52+-+(0.4)*(5.2+9))+(-58)+3.8-81862+79.54/0.76+1.891-(4.2+9.6)/-4.8*3.8/0.2
((7.61*9.6*2.2-0.3/6.1/7.6)*1/6.5/1)*30
+1-+-8.06*++-(7.7/1.7)-5.127/-(+5.1*8.6/5.4-6.0/1.9-6.7-5.7)/27/(1.1/2.1+8.3-7.3)*+-6.5
5-60907.4
-(574-0/-+1.0*0)/(9.4-7.12/32*+3/0.2+2)/++((3.9)/0.6*2)/-+(1.3)*+2/+++0.6*+(7.1)+9.584*-4.8
++-8*-++162*9.11*6.1+(-(2.9/7.9-3.4)*+(2.7)+0*-5.4/+5.4*8.3/1.1)/(698.9)+(-(8.6+6.3)/-2.3*9+++7.9/7.3/9.1-+0.6/9.4/0.8+2.8)+0.052
-(((3.5)/9.6/4.5)-+(8-3)+(9.2-9.0)*8+(4.1)/3.7*9.7+7.2*2.3+7.5+9.7)/3.50+92.69*--7*9++370--(3*1.5*0.0-7.6*7.2)/0/-+4.9*7/+4.0*4.6/1.2-8.814-6*30--4*+5.3*9.6*5.7+9/9.5*8.4+9.6*5.6
0.59
-523
++(1.238*+(0.0)-++7.5)*9/-2+(415.86/519*-8*+3+64.25/(3)*4*2.4--(5.2)/7.5)*+(--9-2.6*9.9*8.0-4.6*0.7)/+-(3.5/5.2-4.5)++0.4*((8.7)+-2.4)*((3.0)/6.1)*-(9.5+2.3)+-+(1-9.4)*--(5.8+3.2)++--4.4*(2*0.1+8.2+4.3)/-3/9-1*-9/3++8.3*5.6*0.6*0.5


In [90]:
class PyCompiledTester(Tester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} testers/grammar_producer_pycompiled.py {seed} {self.max_num} {max_depth} > {fn}"

In [91]:
PyCompiledTester().run_test().show()

depth= 8 size= 57874 time= 0.056 stdev(0.006) throughput= 1015.0678585737179 stdev(115)
depth= 16 size= 249101.5 time= 0.111 stdev(0.001) throughput= 2201.620525888104 stdev(45)
Throughput of  2201.620525888104  kilobytes per second at depth =  16
Total time: 0:00:09.354688


In [92]:
class PyRecCompiledFuzzer(PyCompiledFuzzer):
    def __init__(self, grammar):
        super().__init__(grammar)
        assert self.ordered_grammar
        self.rec_cost = {}
        self.compute_rule_recursion()

    def kr_to_s(self, key, i): return 'gen_%s_%d' % (self.k_to_s(key), i)
    # the grammar needs to be ordered by the cost.
    # else the ordering will change at the end.
    
    def is_rule_recursive(self, rname, rule, seen):
        if not rule: return False
        if rname in seen:
            return False # reached another recursive rule without seeing this one
        for token in rule:
            if token not in self.grammar: continue
            for i,trule in enumerate(self.grammar[token]):
                rn = self.kr_to_s(token, i)
                if rn  == rname: return True
                if rn in seen: return False
                v = self.is_rule_recursive(rname, trule, seen | {rn})
                if v: return True
        return False
    
    def is_key_recursive(self, check, key, seen):
        if not key in self.grammar: return False
        if key in seen: return False
        for rule in self.grammar[key]:
            for token in rule:
                if token not in self.grammar: continue
                if token == check: return True
                v = self.is_key_recursive(check, token, seen | {token})
                if v: return True
        return False
    
    def compute_rule_recursion(self):
        if IS_HTML:   # TODO -- to much time -- only for HTML
            self.rule_recursion = HTML_RULE_RECURSION
            self.key_recursion = HTML_KEY_RECURSION
            return
        self.rule_recursion = {}
        for k in self.grammar:
            for i_rule,rule in enumerate(self.grammar[k]):
                n = self.kr_to_s(k, i_rule)
                self.rule_recursion[n] = self.is_rule_recursive(n, rule, set())
        self.key_recursion = {}
        for k in self.grammar:
            self.key_recursion[k] = self.is_key_recursive(k, k, set())

In [93]:
PyRecCompiledFuzzer(pyc_grammar).rule_recursion

{'gen_start_0': False,
 'gen_expr_0': True,
 'gen_expr_1': True,
 'gen_expr_2': True,
 'gen_term_0': True,
 'gen_term_1': True,
 'gen_term_2': True,
 'gen_factor_0': False,
 'gen_factor_1': False,
 'gen_factor_2': True,
 'gen_factor_3': True,
 'gen_factor_4': True,
 'gen_integer_0': False,
 'gen_integer_1': True,
 'gen_digit_0': False,
 'gen_digit_1': False,
 'gen_digit_2': False,
 'gen_digit_3': False,
 'gen_digit_4': False,
 'gen_digit_5': False,
 'gen_digit_6': False,
 'gen_digit_7': False,
 'gen_digit_8': False,
 'gen_digit_9': False}

In [94]:
PyRecCompiledFuzzer(pyc_grammar).key_recursion

{'<start>': False,
 '<expr>': True,
 '<term>': True,
 '<factor>': True,
 '<integer>': True,
 '<digit>': False}

### Partial Evaluation in Python

In [95]:
class PEFuzzer(PyRecCompiledFuzzer):
    def pe_rule(self, rule, depth):
        res = []
        res.append('''\
# rule=%(rule)s len[%(len)d]''' % {'rule':str(rule), 'len':len(rule)})
        if not rule:
            res.append('''\
pass''')
        for token in rule:
            res.append('''\
# token=%(token)s''' % {'token':token})
            if token in self.grammar:
                if self.key_recursion[token]:
                    res.append('''\
# sc
gen_%(key)s(depth+%(depth)s, max_depth)''' % {'key':self.k_to_s(token),'depth':depth+2})
                else:
                    res.append(self.pe_key(token, depth=depth+1))
            else:
                res.append('''\
result.append("%s")''' % self.esc(token))
        return '\n'.join(res)
 
    def pe_key(self, key, depth):
        if depth == self.MAX_PE_DEPTH:
            return 'gen_%(key)s(depth+%(depth)d, max_depth)' % {'key':self.k_to_s(key), 'depth':depth+1}
        rules = self.grammar[key]
        result = ['''\
#* %(key)s begins
if depth + %(depth)d > max_depth:
    result.append(random.choice(pool_of_strings['%(key)s']))
else:''' % {'name':self.k_to_s(key), 'key':key,  'depth':depth+1}]
        if len(rules) == 0:
            result.append('''\
    # indent dummy''')
        elif len(rules) == 1:
            result.append('''\
    # indent inline''')
            result.append(self.add_indent(self.pe_rule(rules[0], depth),'    '))
        else:
            result.append('''\
    val = random.randrange(%(nrules)s)''' % {'nrules':len(rules)})
            result.append('''\
    # indent here<
    if False:
        pass # dummy''')
            assert len(rules) > 1
            for i, rule in enumerate(rules):
                result.append('''\
    elif val == %d:''' % i)               
                result.append(self.add_indent(self.pe_rule(rule, depth),'        '))
            result.append('''\
    # indent here>''')
        result.append('''\
#* %(key)s ends''' % {'key': key})
        return '\n'.join(result)

    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                if token == key:
                    res.append('''\
# not unrolling
gen_%s(depth+1, max_depth)''' % self.k_to_s(token))
                else:
                    res.append('''\
#indent -<''')
                    res.append(self.pe_key(token, depth=0))
                    res.append('''\
#indent >-''')
                    
            else:
                res.append('''\
result.append("%s")''' % self.esc(token))
        return self.add_indent('\n'.join(res), '            ')

    def string_pool_defs(self):
        result =[]
        result.append('''\
pool_of_strings = %s''' % pp_grammar(self.pool_of_strings))
        result.append('''
result = [];''')
        return '\n'.join(result)

    def gen_main_src(self):
        result = []
        result.append('''
import random
import sys
def main(args):
    global result
    max_num, max_depth = get_opts(args)
    for i in range(max_num):
        gen_start(0, max_depth)
        print(''.join(result))
        result = []
main(sys.argv)
    ''')
        return '\n'.join(result)

    def gen_alt_src(self, key):
        rules = self.grammar[key]
        result = []
        result.append('''
def gen_%(name)s(depth, max_depth):
    # %(name)s begins
    if depth > max_depth:
        result.append(random.choice(pool_of_strings['%(key)s']))
    else:
        val = random.randrange(%(nrules)s)''' % {'name':self.k_to_s(key), 'key':key, 'nrules':len(rules)})
        for i, rule in enumerate(rules):
            result.append('''\
        if val == %d:
%s
            return''' % (i, self.gen_rule_src(rule, key, i)))
        result.append('''
    # %(name)s ends
        ''')
        return '\n'.join(result)
    
    def fuzz_src(self, key='<start>'):
        self.MAX_PE_DEPTH = 4
        result = [self.gen_fuzz_src(),
                  self.gen_main_src()]
        return ''.join(result)

In [96]:
with open('testers/grammar_producer_pe.py', 'w+') as f:
    for fn in [get_opts]:
        print(inspect.getsource(fn), file=f)
    result = PEFuzzer(pyc_grammar).fuzz_src()
    print(result, file=f)

In [97]:
!wc -l testers/grammar_producer_pe.py

    1440 testers/grammar_producer_pe.py


In [98]:
!{sys.executable} testers/grammar_producer_pe.py 0 10 10

+(+-+(8.4)*29.81/20+-+(5.2)*(8.6+1.0))+(-55)+3.8-81880+79.54/0.78+1.862-(0.5+8.2)/-4.9*4.3/6.2
((0.82*5.7*9-4.9/1.0/3.1)*1/6.5/4)*30
+1-+-8.06*++-(9.4/9)-5.127/-(+1*4.0/1.0-7.6/8-5.9-5.5)/27/(6.0/8.2+0.9-0.8)*+-4.8
5-60907.4
-(573-0/-+8.4*4)/(9.4-7.11/70*+4.6/4.1+1)/++((5.7)/9.6*6.4)/-+(7.8)*+2/+++9.6*+(5.3)+9.584*-4.8
++-8*-++161*9.11*6.1+(-(8.2/3.3-7.2)*+(0.4)+0*-3.8/+1.8*8.3/6.8)/(666.9)+(-(7+4.7)/-0.9*5+++9.0/1.9/0.5-+9.6/9.5/9.6+6.8)+0.052
-(((9.8)/2.8/8.1)-+(0.6-0.5)+(2.4-7.6)*6+(4.6)/5.8*7.5+6*0.0+4.0+8.1)/3.50+92.69*--7*9++370--(9*8*2.8-5.3*4.9)/0/-+8.5*0/+2.2*2.1/3.9-8.813-6*74--8*+6.5*2.8*5.7+5/4.4*7.0+2.8*7.9
0.59
-523
++(1.276*+(0.4)-++5.9)*9/-2+(412.86/545*-6*+4.6+63.22/(3.8)*8*6.9--(9.7)/1.3)*+(--2.5-0.7*7.5*2.9-2.1*1.9)/+-(7.4/8.6-8.1)++0.4*((5.6)+-6.9)*((8)/1.0)*-(4.3+6)+-+(0.2-5.5)*--(4.4+5.1)++--8.8*(3.6*6+3.6+6.0)/-9/5-1*-5/9++6.9*3.7*9.6*8.0


In [99]:
class PyPETester(Tester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} testers/grammar_producer_pe.py {seed} {self.max_num} {max_depth} > {fn}"

In [100]:
PyPETester().run_test().show()

depth= 8 size= 57860 time= 0.054 stdev(0.001) throughput= 1046.902136042024 stdev(41)
depth= 16 size= 249097.5 time= 0.107 stdev(0.003) throughput= 2274.653531495741 stdev(91)
Throughput of  2274.653531495741  kilobytes per second at depth =  16
Total time: 0:00:09.396165


### Supercompile in Python

In [101]:
class PySuperCompiledFuzzer(PyRecCompiledFuzzer):    
    def supercompile_rule(self, key, rule, i_rule, depth):
        gen_name = self.kr_to_s(key, i_rule)
        if self.rule_recursion[gen_name]:
            self.current_lst.append(gen_name)
            return '''\
%(gen_name)s(depth_%(depth)d) # recursing''' % {'gen_name':gen_name, 'depth':depth}
        res = []
        if len(rule) == 0:
            res.append('pass')
        else:
            for token in rule:
                if token not in self.grammar:
                    res.append('''\
result.append("%s")''' % self.esc(token))
                else:
                    res.append(# no indent
                        self.supercompile_key(token,
                                              depth=(depth+1)))
        return '\n'.join(res)
    def supercompile_key_internal(self, key, trule, i_trule, depth):
        if depth > self.MAX_SUPERCOMPILE_DEPTH:
            self.current_lst.append(self.kr_to_s(key, i_trule))
            return '%(gen_name)s(depth_%(depth)d) #slimit*' % {
                        'gen_name':self.kr_to_s(key, i_trule), 'depth':depth}
        else:
            return self.supercompile_rule(key, trule, i_trule, depth=depth)
 
    def supercompile_key(self, key, depth):
        # Should check for MAX_SUPERCOMPILE_DEPTH
        # should first get the random number curresponding to
        # len(grammar[key]) then it should unroll that elif cond.
        if len(self.grammar[key]) == 0: return '' # no more jumping on the bed
        res = ['''\
if depth_%(depth)d > max_depth:
    result.append(random.choice(pool_of_%(key)s))
else:
    depth_%(d_1)d = depth + %(d_1)d'''%{
            'key':self.k_to_s(key), 'depth': depth, 'd_1': depth+1}]
        if len(self.grammar[key]) == 1:
            # we do not have to get the random number, and check for
            # equality first.
            i_trule, trule  = 0, self.grammar[key][0]
            res.append(self.add_indent(
                self.supercompile_key_internal(key, trule, i_trule, depth),
                '''\
    '''))
        else:
            # First get the random number, then compare and
            # unroll
            res.append('''\
    val = random.randrange(%(len_rules)d)
    if False: # dummy for elsif
        pass''' % {'len_rules': len(self.grammar[key])})
            for i_trule, trule in enumerate(self.grammar[key]):
                res.append('''\
    elif val == %(i_trule)d:''' % {'i_trule': i_trule})
                res.append(self.add_indent(
                    self.supercompile_key_internal(key, trule, i_trule, depth),
                    '''\
        '''))

        return '\n'.join(res)
   
    def gen_rule_src(self, rule, key, i_rule):
        res = ['''\
def %(gen_name)s(depth):
    if depth > max_depth:
        result.append(random.choice(pool_of_%(key)s))
    else:
        depth_%(d_1)d = depth + %(d_1)d''' % {
            'gen_name':self.kr_to_s(key,i_rule),
            'key':self.k_to_s(key),
            'depth':0, 'd_1': 1}]
        
        # These should be a sequence of getting randon numbers
        # and unrolling appropriately.
        for token in rule:
            if token not in self.grammar:
                res.append('''\
        result.append("%s")''' % self.esc(token))
            else:
                res.append(self.add_indent(
                    self.supercompile_key(token, depth=1), '''\
        '''))
        return '\n'.join(res)

    def gen_main_src(self):
        result = []
        result.append('''
import random
import sys
max_depth = 0
def main(args):
    global result, max_depth
    max_num, max_depth = get_opts(args)
    for i in range(max_num):
        gen_start_0(0)
        print(''.join(result))
        result = []
main(sys.argv)
    ''')
        return '\n'.join(result)
    
    def gen_fuzz_src(self):
        keys_used = {}
        result = [self.string_pool_defs()]
        key_defs = {}
        for key in self.grammar:
            for i,rule in enumerate(self.grammar[key]):
                self.current_lst = []
                ks = self.kr_to_s(key, i)
                keys_used[ks] = self.current_lst
                key_defs[ks] = self.gen_rule_src(rule, key, i)
        key_set = set(keys_used['gen_start_0']) | {'gen_start_0'}
        old_len = 0
        while old_len != len(key_set):
            old_len = len(key_set)
            key_set.update(k1 for k in list(key_set) for k1 in keys_used[k])
            
        for k in key_set:
            result.append(key_defs[k])
        return '\n'.join(result)

    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 100
        result = [self.gen_fuzz_src(),
                  self.gen_main_src()]
        return ''.join(result)

In [102]:
with open('testers/grammar_producer_pysupercompiled.py', 'w+') as f:
    for fn in [get_opts]:
        print(inspect.getsource(fn), file=f)
    result = PySuperCompiledFuzzer(pyc_grammar).fuzz_src()
    print(result, file=f)

In [103]:
!wc -l testers/grammar_producer_pysupercompiled.py

     909 testers/grammar_producer_pysupercompiled.py


In [104]:
!{sys.executable} testers/grammar_producer_pysupercompiled.py 0 10 10

86798*+9.84+75/+((4.4+8.9)-(3.9)+2.1/8.2)*55/9/8.17
8189*(54*0.75+1.825-(4.9+6.8)/-5.9*6.4/0.6)/97*+0.15/+-1-0-6.5/3
30/-(1*++0.9/2)/6*++-(6.1)
2-+27/-(+-2.1*+3.9/1.3/0.2-0/2.7/7.5+5.5+8.7+8.5)/+(9.1/4.0/5.4-6.9-6.4-3.2)*3.6031*7/+4.2+(-++2.1)/32*--+5+3/(+6.8-8.8-4.2+7.1)
74-(+4/3-(1.7*5.0+5.0)*(5.5-1.4)/-6.2/5*6--(8.6))/4.5190
584.7+24-+-8*-++54*52*(9.1/8.0+1.3)/0.5/(8.9)*5.8*8.8+1390.5/8-7.40/(3.4)++(0.1)*-3.1/-4.7/1.5*9.9+4.70*(5.9)+1.3/7.4*3.9+4.3+1.2
0.052
+((2/6.5/1.9)/891.0*6/(7.6)/4.2*5.5+---7.7*6.48+36.7*3*9.7*1.4+-6.3)*9-+370--(11*2*2.8-+1*6.0+6.5/9.2+8.8)/3*(-9.3/6.1*5.1-3.7/9+9.0)--7.67
+15/50/+3+4
2/-523+-++(3.6*8.0-7.8*1.0-9.1)*+(71.1)+-2-(60.8/+6.2*7.7/1.0+-9.9+7.6*7+3.8+6.9)*17+--9.27+344*-5/9*1.0/2.2+-4/(9)+6*7.9*9.2+3.2*8.1


In [105]:
class PySupercompiledTester(Tester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"{sys.executable} testers/grammar_producer_pysupercompiled.py {seed} {self.max_num} {max_depth} > {fn}"

In [106]:
PySupercompiledTester().run_test().show()

depth= 8 size= 57541 time= 0.047 stdev(0.007) throughput= 1207.8061505551739 stdev(162)
depth= 16 size= 260687.5 time= 0.113 stdev(0.013) throughput= 2265.152881906132 stdev(218)
Throughput of  2265.152881906132  kilobytes per second at depth =  16
Total time: 0:00:09.107084


### Compile to C

In [107]:
class CTrans(Sanitize):
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        esc = {
           '\r': '\\r',
           '\n': '\\n',
           '\t': '\\t',
           '\\': '\\\\',
        }
        for i in t:
            #if i in esc:
            #    my_tokens.append(esc[i])
            #else:
                my_tokens.append(i)
        return my_tokens

In [108]:
c_grammar = CTrans(my_grammar).translate()

In [109]:
c_grammar

{'<start>': [['<expr>']],
 '<expr>': [['<term>', '+', '<expr>'], ['<term>', '-', '<expr>'], ['<term>']],
 '<term>': [['<factor>', '*', '<term>'],
  ['<factor>', '/', '<term>'],
  ['<factor>']],
 '<factor>': [['+', '<factor>'],
  ['-', '<factor>'],
  ['(', '<expr>', ')'],
  ['<integer>', '.', '<integer>'],
  ['<integer>']],
 '<integer>': [['<digit>', '<integer>'], ['<digit>']],
 '<digit>': [['0'],
  ['1'],
  ['2'],
  ['3'],
  ['4'],
  ['5'],
  ['6'],
  ['7'],
  ['8'],
  ['9']]}

We compile a grammar into a C program that produces from the grammar.  Just how fast can we be?

In [110]:
class CFuzzer(PyRecCompiledFuzzer):    
    def cheap_chars(self, string):
        # to be embedded within single quotes
        escaped = {'t':'\t', 'n': '\n', "'": "\\'", "\\":"\\\\", 'r': '\r'}
        slst = []
        while string:
            c, *string = string
            if c in {'\\'}:
                c1, *string = string
                slst.append(escaped[c1])
            elif c in {"'"}:
                slst.append("\'")
            else:
                slst.append(c)
        return slst
    
    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                res.append('gen_%s(depth +1);' % self.k_to_s(token))
            else:
                res.append("out('%s');" % self.esc_char(token))
        return '\n        '.join(res)

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        cheap_strings = self.pool_of_strings[k]
        result = ['''
void gen_%(name)s(int depth) {
    if (depth > max_depth) {
        int val = map(%(num_cheap_strings)d);
        const char* str = pool_%(name)s[val];
        const int str_l = pool_l_%(name)s[val];
        for (int i = 0; i < str_l; i++) {
            out(str[i]);
        }
        return;
    }

    int val = map(%(nrules)d);
    switch(val) {''' % {'name':self.k_to_s(k), 'nrules':len(rules),
                        'num_cheap_strings': len(cheap_strings),
                       }]
        for i, rule in enumerate(rules):
            result.append('''
    case %d:
        %s
        break;''' % (i, self.gen_rule_src(rule, k, i)))
        result.append('''
    }
}
    ''')
        return '\n'.join(result)
    
    def string_pool_defs(self):
        result = []
        for k in self.grammar:
            cheap_strings = self.pool_of_strings[k]
            result.append('''
const char* pool_%(k)s[] =  {%(cheap_strings)s};
const int pool_l_%(k)s[] =  {%(cheap_strings_len)s};
        ''' % {'k':self.k_to_s(k),
               'cheap_strings': ', '.join(['"%s"' % self.esc(s) for s in cheap_strings]),
               'cheap_strings_len': ', '.join([str(len(s)) for s in cheap_strings])})
        return '\n'.join(result)

    
    def fn_fuzz_decs(self):
        result = []
        for k in self.grammar:
            result.append('''void gen_%s(int depth);''' % self.k_to_s(k))
        return '\n'.join(result)
    
    def fn_map_def(self):
        return '''
int map(int v) {
    return random() % v;
}
 '''    
    def fn_out_def(self):
        return '''
void out(const char s) {
    fputc(s, stdout);
}       
 '''

    def fuzz_hdefs(self):
        return '''
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>
'''
    
    def fuzz_out_var_defs(self):
        return '''
void out(const char s);'''
    
    def fuzz_rand_var_defs(self):
        return '''
int map(int v);'''
    def fuzz_stack_var_defs(self):
        return '''
extern int max_depth;'''

    def fuzz_var_defs(self):
        return '\n'.join([self.fuzz_out_var_defs(), self.fuzz_rand_var_defs(), self.fuzz_stack_var_defs()])

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''
    
    def fn_main_loop_frag(self):
        return '''
    for(int i=0; i < max_num; i++) {
        gen_init__();
    }'''

    def fn_main_def(self):
        result = '''
int main(int argc, char** argv) {
    int seed, max_num;
%(input_frag)s
    //srandom(time(0));
    srandom(seed);
%(loop_frag)s
    return 0;
}''' % {'input_frag':self.fn_main_input_frag(),
        'loop_frag': self.fn_main_loop_frag()}
        return result
    
    def main_stack_var_defs(self):
        return '''
int max_depth = 0;'''
    
    def main_init_var_defs(self):
        return '''
void gen_init__();'''
    
    def main_var_defs(self):
        return '\n'.join([self.main_stack_var_defs(), self.main_init_var_defs()])
    
    def fuzz_fn_defs(self):
        result = []
        for key in self.grammar:
            result.append(self.gen_alt_src(key))
        return '\n'.join(result)
    
    def fuzz_entry(self):
        return '''
void gen_init__() {
    gen_start(0);
    out('\\n');
    return;
}'''

    def main_hdefs(self):
        return '''
#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>
'''

    def gen_main_src(self):
        return '\n'.join([self.main_hdefs(),
                          self.main_var_defs(),
                          self.fn_map_def(),
                          self.fn_out_def(),
                          self.fn_main_def()])
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          self.fuzz_fn_defs(),
                          self.fuzz_entry()])

    def fuzz_src(self, key='<start>'):
        return self.gen_main_src(), self.gen_fuzz_src()

In [111]:
main_src, fuzz_src = CFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_c_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/grammar_producer_c_main.c', 'w+') as f:
    print(main_src, file=f)

In [112]:
!nl -ba testers/grammar_producer_c_fuzz.c

     1	
     2	#include <stdlib.h>
     3	#include <stdio.h>
     4	#include <time.h>
     5	#include <string.h>
     6	
     7	
     8	void out(const char s);
     9	
    10	int map(int v);
    11	
    12	extern int max_depth;
    13	void gen_start(int depth);
    14	void gen_expr(int depth);
    15	void gen_term(int depth);
    16	void gen_factor(int depth);
    17	void gen_integer(int depth);
    18	void gen_digit(int depth);
    19	
    20	const char* pool_start[] =  {"1.0", "8.7", "1.9", "6", "4", "7.7", "0.6", "5.7", "8.6", "3.0", "6.9", "8.3", "2.2", "0.7", "1.6", "4.0", "1.7", "7.4", "3.8", "5.2", "1.3", "3.7", "1.4", "5.1", "3.3", "7.5", "5.3", "0.1", "8.0", "8.1", "5.8", "9.8", "6.1", "1.1", "6.8", "7.8", "4.3", "5.4", "2.9", "7.9", "2.1", "3.4", "6.7", "5", "3.5", "3.6", "2.8", "0.4", "9.5", "2.7", "3", "5.0", "3.1", "1.2", "5.5", "4.5", "0.8", "7.3", "8.8", "5.9", "9.2", "8.2", "9.0", "7.1", "9.1", "4.9", "6.4", "0.5", "4.8", "9", "2.5", "4.4", "3.9", "3.2", "6.0", "2", "4.

In [113]:
!cat testers/grammar_producer_c_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>


int max_depth = 0;

void gen_init__();

int map(int v) {
    return random() % v;
}
 

void out(const char s) {
    fputc(s, stdout);
}       
 

int main(int argc, char** argv) {
    int seed, max_num;

    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);
    //srandom(time(0));
    srandom(seed);

    for(int i=0; i < max_num; i++) {
        gen_init__();
    }
    return 0;
}


In [114]:
%cd testers
!cc -Ofast grammar_producer_c_main.c grammar_producer_c_fuzz.c  -o grammar_producer_c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [115]:
!./testers/grammar_producer_c 0 10 10

52*316-98.9*75.1
3.4+(9029*9.6*3.64*(9.4)/7.6++(3.7)*-(8.3)/8*2.2/3.5+7.61+(3.1)/7.1-6.2*3.0+6+1)-8.4/(88+-4.6)
(158.28---+(7.3)+--(9.0)*58.5/-3.5-5-(7.4)*6.8*4.6+6.5*7.5)
766
-(0.74*(4.7*3.1)+++-7+4.85*-2.7/3)-+((6.6-0.3)/23*-2.4/4.1++(1)+-8.9/0.6/6.7+0.5+1.1-2.9)-+7.028*47*(+7.5/2/0.7)/(3.2+1.6+8.0)/-2*(6.4)/8.0*8.1++--8.4/02.8+(4+7.2*4.7+3.1-4.7)*73.56/(7.4)-120.156/+4.5+3-8.1/0.5/3.1+0*8.6+1.1
-511621
+29*-(+(6.6-9.7)*-+0.4--+8)*2
-(3-(2.2+8.2)+3.4*-2.4*7.4)/5*--((1.8)/4.8/8.7-3.7*6.6+4.1)-(+461.50-554.24+87.7)*-+-++-7.4*--4.525/7.8*6.550/++2.9*(7.8)*1.1
--39*37.4/2.7
73


In [116]:
# II
class CTester(Tester):
    def __init__(self, name=None, max_num=1000, start_depth=3, limit_depth=5, timeout=3600, iterations=100):
        super().__init__(name, max_num, start_depth, limit_depth, timeout)

    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_c {seed} {self.max_num} {max_depth} > {fn}"

In [117]:
CTester().run_test().show()

depth= 8 size= 57399.5 time= 0.005 stdev(0.0) throughput= 11210.83984375 stdev(377)
depth= 16 size= 259013 time= 0.014 stdev(0.006) throughput= 19318.161492598683 stdev(8539)
Throughput of  19318.161492598683  kilobytes per second at depth =  16
Total time: 0:00:08.503909


### Partial Evaluation in C

In [118]:
class PECFuzzer(CFuzzer):
    def pe_rule(self, rule, seen, depth):
        res = []
        res.append('''\
/* rule=%(rule)s len[%(len)d]*/''' % {'rule': str(rule), 'len': len(rule)})
        if not rule:
            res.append('''
/*break;*/
            ''')
        for token in rule:
            res.append('''
/* token=%(token)s*/''' % {'token': token})
            if token in self.grammar:
                if token in seen:
                    res.append('''\
/* sc */
gen_%(key)s(depth+%(depth)d);''' % {'key': self.k_to_s(token), 'depth': depth+2})
                else:
                    res.append(self.pe_key(token, seen=(seen | {token}), depth=depth+1))
            else:
                res.append('''\
out('%s');''' % self.esc_char(token))
        return '\n'.join(res)
        
        
    def pe_key(self, key, seen, depth):
        if depth == self.MAX_PE_DEPTH:
            return 'gen_%(key)s(depth+%(depth)d);' % {'key': self.k_to_s(key), 'depth': depth+1}
        rules = self.grammar[key] # ordered by the cost
        cheap_strings = self.pool_of_strings[key] 
        # we haven't restricted map to 256 yet.
        result = ['''\
/* %(key)s begins*/
if ((depth + %(depth)d) > max_depth) {
    int val = map(%(num_cheap_strings)d);
    const char* str = pool_%(name)s[val];
    const int str_l = pool_l_%(name)s[val];
    for (int i = 0; i < str_l; i++) {
        out(str[i]);
    }
} else {''' %  {'name':self.k_to_s(key),
                'key': key,
                'depth': depth+1,
                'num_cheap_strings': len(cheap_strings)}]
        if len(rules) == 0:
            result.append('''\
    /*indent dummy*/
            ''')
        elif len(rules) == 1:
            result.append('''\
    /*indent inline*/
            ''')
            result.append(self.add_indent(self.pe_rule(rules[0], seen, depth),'    '))
        else:
            result.append('''\
    int val = map(%(nrules)d);
            ''' % {'nrules': len(rules)})
            result.append('''\
    switch(val) {
            ''')
            for i, rule in enumerate(rules):
                result.append('''\
    case %d:
        {''' % i)
                result.append(self.add_indent(self.pe_rule(rule, seen, depth),'            '))
                result.append('''\
            break;
        }''')
            result.append('''\
    }''')
        result.append('''\
}
/* %(key)s ends*/''' % {'key': key})
        return '\n'.join(result)
        
    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                if token == key:
                    res.append('''\
/* not unrolling*/
gen_%(key)s(depth +1);''' % {'key':self.k_to_s(token)})
                else:
                    res.append('''\
/*indent -<*/''')
                    res.append(self.pe_key(token, seen={key, token}, depth=0))
                    res.append('''\
/*indent >-*/''')
            else:
                res.append("out('%s');" % self.esc_char(token))
        return '\n'.join(res)
    
    def fuzz_src(self, key='<start>'):
        self.MAX_PE_DEPTH = 4
        return self.gen_main_src(), self.gen_fuzz_src()

In [119]:
main_src, fuzz_src = PECFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_pec_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/grammar_producer_pec_main.c', 'w+') as f:
    print(main_src, file=f)

In [120]:
!wc -l testers/grammar_producer_pec_fuzz.c

   12563 testers/grammar_producer_pec_fuzz.c


In [121]:
!cat testers/grammar_producer_pec_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>


int max_depth = 0;

void gen_init__();

int map(int v) {
    return random() % v;
}
 

void out(const char s) {
    fputc(s, stdout);
}       
 

int main(int argc, char** argv) {
    int seed, max_num;

    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);
    //srandom(time(0));
    srandom(seed);

    for(int i=0; i < max_num; i++) {
        gen_init__();
    }
    return 0;
}


In [122]:
%cd testers
!cc -Ofast grammar_producer_pec_main.c grammar_producer_pec_fuzz.c  -o grammar_producer_pec
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [123]:
!./testers/grammar_producer_pec 0 10 10

52*316-98.9*75.1
3.4+(9019*9.6*8.96*(5.4)/8.6++(0.6)*-(7.5)/8*2.2/7.8+7.94+(4.5)/1.3-9.5*2.8+4.7+2.6)-8.4/(68+-5.4)
(158.28---+(2.4)+--(3.1)*58.5/-5.9-5-(4)*6.3*4.5+8.2*3)
766
-(0.74*(7.1*5)+++-2.0+4.67*-4.6/4.7)-+((3.6-0.1)/10*-9/4.0++(2.6)+-1.4/5.0/9+3.5+7-5.0)-+7.028*47*(+5.3/6.4/6.3)/(1.6+0.7+8.9)/-1*(7.2)/2.5*1++--6.4/02.8+(6+7.8*5.0+5-9.4)*78.59/(5.2)-115.153/+6.7+8-8.4/3.5/5+0.3*4+7
-511620
+29*-(+(3.9-1.3)*-+3.1--+6.7)*2
-(3-(2.2+9.0)+8.4*-9*4.4)/5*--((6.9)/7.3/8.6-3.4*3.9+9.9)-(+494.53-556.24+62.7)*-+-++-4.4*--4.517/7.8*6.555/++0.9*(9.9)*6.8
--39*37.4/2.7
73


In [124]:
class CPETester(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_pec {seed} {self.max_num} {max_depth} > {fn}"

In [125]:
CPETester().run_test().show()

depth= 8 size= 57391.5 time= 0.007 stdev(0.0) throughput= 8006.626674107142 stdev(268)
depth= 16 size= 258945 time= 0.011 stdev(0.001) throughput= 22027.776544744316 stdev(1254)
Throughput of  22027.776544744316  kilobytes per second at depth =  16
Total time: 0:00:08.830684


### Supercompile C

In [126]:
# II
class CSuperCompiledFuzzer(CFuzzer):

    def kr_to_s(self, key, i): return 'gen_%s_%d' % (self.k_to_s(key), i)
    
    def supercompile_rule(self, key, rule, i_rule, depth):
        gen_name = self.kr_to_s(key, i_rule)
        if self.rule_recursion[gen_name]:
            self.current_lst.append(gen_name)
            return '''\
%(gen_name)s(depth_%(depth)d); /* recurse*/''' % {'gen_name':gen_name, 'depth':depth}
        res = []
        if len(rule) == 0:
            res.append('/*pass*/')
        else:
            for token in rule:
                if token not in self.grammar:
                    res.append('''\
out('%s');''' % self.esc_char(token))
                else:
                    res.append(# no indent
                        self.supercompile_key(token,
                                              depth=(depth+1)))
        return '\n'.join(res)
    
    def supercompile_key_internal(self, key, trule, i_trule, depth):
        if depth > self.MAX_SUPERCOMPILE_DEPTH:
            self.current_lst.append(self.kr_to_s(key, i_trule))
            return '%(gen_name)s(depth_%(depth)d); /*slimit*/' % {
                        'gen_name':self.kr_to_s(key, i_trule), 'depth':depth}
        else:
            return self.supercompile_rule(key, trule, i_trule, depth=depth)
        
    def choose_from_cheap_strings(self, key):
        cheap_strings = self.pool_of_strings[key]
        if len(cheap_strings) == 1:
            return '\n'.join(["out('%s');" % c for c in self.cheap_chars(cheap_strings[0])])
        l = [len(s) for s in cheap_strings]
        if len(set(l)) == 1:
            name = ['''\
int val = map(%(num_cheap_strings)d);
const char* str = pool_%(name)s[val];''' % {
                'name':self.k_to_s(key), 'num_cheap_strings': len(cheap_strings)}]
            out = ["out(str[%d]);" % i for i in range(l[0])]
            return '\n'.join(name + out)
        else:
            return '''\
int val = map(%(num_cheap_strings)d);
const char* str = pool_%(name)s[val];
const int str_l = pool_l_%(name)s[val];
for (int i = 0; i < str_l; i++) {
    out(str[i]);
}
    '''%{
            'name':self.k_to_s(key),
            'num_cheap_strings': len(cheap_strings)}

    def supercompile_key(self, key, depth):
        # Should check for MAX_SUPERCOMPILE_DEPTH
        # should first get the random number curresponding to
        # len(grammar[key]) then it should unroll that elif cond.
        if len(self.grammar[key]) == 0: return '' # no more jumping on the bed

        res = ['''\
if (depth_%(depth)d > max_depth) {
%(select_from_pool)s
} else {
    int depth_%(d_1)d = depth + %(d_1)d;'''%{
            'select_from_pool': self.add_indent(self.choose_from_cheap_strings(key), '    '),
            'depth': depth,
            'd_1': depth+1}]
        if len(self.grammar[key]) == 1:
            # we do not have to get the random number, and check for
            # equality first.
            i_trule, trule  = 0, self.grammar[key][0]
            res.append(self.add_indent(
                self.supercompile_key_internal(key, trule, i_trule, depth),
                '''\
    '''))
        else:
            # First get the random number, then compare and
            # unroll
            res.append('''\
    int val = map(%(len_rules)d);
    switch(val) {''' % {'len_rules': len(self.grammar[key])})
            for i_trule, trule in enumerate(self.grammar[key]):
                res.append('''\
    case %(i_trule)d:
        {''' % {'i_trule': i_trule})
                res.append(self.add_indent(
                    self.supercompile_key_internal(key, trule, i_trule, depth),
                    '''\
            '''))
                res.append('''\
            break;
        } /*case %d*/''' % i_trule)
            res.append('''\
    }/*switch*/''')
        res.append('''\
}/*ifelse %d*/''' % depth)
        return '\n'.join(res)
   
    def gen_rule_src(self, rule, key, i_rule):
        res = ['''\
void %(gen_name)s(int depth) {
    if (depth > max_depth) {
%(select_from_pool)s
    } else {
        int depth_%(d_1)d = depth + %(d_1)d; ''' % {
            'gen_name':self.kr_to_s(key,i_rule),
            'select_from_pool': self.add_indent(self.choose_from_cheap_strings(key),'        '),
            'depth':0,
            'd_1': 1}]
        # These should be a sequence of getting randon numbers
        # and unrolling appropriately.
        for token in rule:
            if token not in self.grammar:
                res.append('''\
        out('%s');''' % self.esc_char(token))
            else:
                res.append(self.add_indent(
                    self.supercompile_key(token, depth=1), '''\
        '''))
        res.append('''\
    } /*else*/
} /* %s */''' % self.kr_to_s(key, i_rule))
        return '\n'.join(res)
    
    # ----  
 
    def fn_fuzz_decs(self):
        result = []
        for k in self.grammar:
            for i,r in enumerate(self.grammar[k]):
                result.append('void %(name)s(int depth);' % {'name':self.kr_to_s(k, i)})
        return '\n'.join(result)
    
    def fuzz_fn_defs(self):
        keys_used = {}
        result = []
        key_defs = {}
        for key in self.grammar:
            for i_rule,rule in enumerate(self.grammar[key]):
                self.current_lst = []
                ks = self.kr_to_s(key, i_rule)
                keys_used[ks] = self.current_lst
                key_defs[ks] = self.gen_rule_src(rule, key, i_rule)
        key_set = set(keys_used['gen_start_0']) | {'gen_start_0'}
        old_len = 0
        while old_len != len(key_set):
            old_len = len(key_set)
            key_set.update(k1 for k in list(key_set) for k1 in keys_used[k])
            
        for k in key_set:
            result.append(key_defs[k])
        return '\n'.join(result)
 
    def fuzz_entry(self):
        return '''
void gen_init__() {
    gen_start_0(0);
    out('\\n');
    return;
}'''
 
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 0
        return self.gen_main_src(), self.gen_fuzz_src()

Going full supercompilation (below) actually reduces the speed by a small %. It is not clear why.

In [127]:
class CSuperCompiledFuzzer(CSuperCompiledFuzzer):
    def string_pool_defs(self): return ''
    
    def choose_from_cheap_strings(self, key):
        short = False
        cheap_strings = self.pool_of_strings[key]
        if len(cheap_strings) == 1:
            return '\n'.join(["out('%s');" % self.esc_char(c) for c in cheap_strings[0]])
        elif len(cheap_strings) == 0:
            return ''
        else:
            lst = ['''
int val = map(%(num_cheap_strings)d);
switch(val){'''% {
            'name':self.k_to_s(key),
            'num_cheap_strings': len(cheap_strings)}]
            for i in range(len(cheap_strings)):
                lst.append('''
case %d:
    {''' % i)
                lst.extend(["    out('%s');" % self.esc_char(c) for c in  cheap_strings[i]])
                lst.append('''
    break;
    }''')
            lst.append('''
}''')
            return '\n'.join(lst)

In [128]:
main_src, fuzz_src = CSuperCompiledFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_superc_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/grammar_producer_superc_main.c', 'w+') as f:
    print(main_src, file=f)

In [129]:
!wc -l testers/grammar_producer_superc_fuzz.c

   27446 testers/grammar_producer_superc_fuzz.c


In [130]:
!cat testers/grammar_producer_superc_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>


int max_depth = 0;

void gen_init__();

int map(int v) {
    return random() % v;
}
 

void out(const char s) {
    fputc(s, stdout);
}       
 

int main(int argc, char** argv) {
    int seed, max_num;

    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);
    //srandom(time(0));
    srandom(seed);

    for(int i=0; i < max_num; i++) {
        gen_init__();
    }
    return 0;
}


In [131]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [132]:
!./testers/grammar_producer_superc 0 10 10

-52/316
98.9*75.1
6.42*(9042*9.6*7.20*(2.0)/8.9++(9.1)*-(8.7)/5*9.3/3.6+0.28+(1.1)/6.9-7.4*5+6.9+6.5)
8.4/(68+(3.5+9.3+9.9)*36)
(-3259.3)+-((3.2-4.4))/((2.8*6.3)*1*(2.9))
+-766/-4.3*+74*((9.3)*+4.7)-++3.60
-(+(3.8/9.6+6.9)--47/-+8.6/5)-++--(1*9.6-5.5)/72.6/(++6.7--3.8/0.2*2.9+8.6+0.1+3)*4+--90+((7.1+6.9)*-0.6-2.7*2+6.3+7.2+6.1)/-(3.1+6.0)/6.1*08.76/7+-6.149
-(+6.016+4.5/56*3.2+71/(8.7)/1)*++(++1.6)*++((2.8)*5.1*1.9+7.6/1.9-9.2)
(++2/-(2.1-8.4)*-1.8)-(5.3*-+3*7*-5.2)
53886/4/1401.04/545.2


In [133]:
class CSupercompiledTester(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_superc {seed} {self.max_num} {max_depth} > {fn}"

In [134]:
CSupercompiledTester().run_test().show() # 0

depth= 8 size= 58751.5 time= 0.005 stdev(0.001) throughput= 11954.305013020832 stdev(3390)
depth= 16 size= 260194 time= 0.02 stdev(0.0) throughput= 12704.78515625 stdev(410)
Throughput of  12704.78515625  kilobytes per second at depth =  16
Total time: 0:00:08.735929


In [135]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 1
        return self.gen_main_src(), self.gen_fuzz_src()

In [136]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [137]:
CSupercompiledTester().run_test().show() # 1

depth= 8 size= 58751.5 time= 0.007 stdev(0.001) throughput= 7683.811732700893 stdev(719)
depth= 16 size= 260194 time= 0.02 stdev(0.001) throughput= 12395.39271763393 stdev(28)
Throughput of  12395.39271763393  kilobytes per second at depth =  16
Total time: 0:00:09.102638


In [138]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 2
        return self.gen_main_src(), self.gen_fuzz_src()

In [139]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [140]:
CSupercompiledTester().run_test().show() # 2

depth= 8 size= 58751.5 time= 0.007 stdev(0.001) throughput= 8879.022507440477 stdev(959)
depth= 16 size= 260194 time= 0.014 stdev(0.001) throughput= 17530.908203125 stdev(290)
Throughput of  17530.908203125  kilobytes per second at depth =  16
Total time: 0:00:08.837692


In [141]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 3
        return self.gen_main_src(), self.gen_fuzz_src()

In [142]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [143]:
CSupercompiledTester().run_test().show() # 3

depth= 8 size= 58751.5 time= 0.005 stdev(0.003) throughput= 13663.52771577381 stdev(7738)
depth= 16 size= 260194 time= 0.015 stdev(0.002) throughput= 16511.733111213234 stdev(1731)
Throughput of  16511.733111213234  kilobytes per second at depth =  16
Total time: 0:00:08.802468


In [144]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 5
        return self.gen_main_src(), self.gen_fuzz_src()

In [145]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [146]:
CSupercompiledTester().run_test().show() # 5

depth= 8 size= 58751.5 time= 0.005 stdev(0.001) throughput= 12910.0341796875 stdev(2038)
depth= 16 size= 260194 time= 0.017 stdev(0.006) throughput= 15638.977477600525 stdev(6155)
Throughput of  15638.977477600525  kilobytes per second at depth =  16
Total time: 0:00:08.820562


In [147]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 10
        return self.gen_main_src(), self.gen_fuzz_src()

In [148]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [149]:
CSupercompiledTester().run_test().show() # 10

depth= 8 size= 58751.5 time= 0.005 stdev(0.002) throughput= 11268.345424107143 stdev(4338)
depth= 16 size= 260194 time= 0.014 stdev(0.003) throughput= 18588.165283203125 stdev(4341)
Throughput of  18588.165283203125  kilobytes per second at depth =  16
Total time: 0:00:08.585593


## Making the random -> choices map faster.

### [Replace the division by mulitplication](//https://lemire.me/blog/2016/06/27/a-fast-alternative-to-the-modulo-reduction/)

For convenience, here is our map function.
```c
int map(int v) {
    return random() % v;
}
```

```c
uint32_t
__attribute__((always_inline))
map(uint32_t to) {
    uint32_t from = random();
    return ((uint64_t) from * (uint64_t) to) >> 32;
}
```

### Replace random() by a faster pseudo random.

[Xoshiro**](http://xoshiro.di.unimi.it/xoshiro128starstar.c)

In [150]:
# https://lemire.me/blog/2016/06/27/a-fast-alternative-to-the-modulo-reduction/ 
class CFuzzerPRNG(CFuzzer):
    def fn_map_def(self):
        return '''
uint64_t next(void);
uint32_t map(uint32_t to) {
    uint32_t from = next();
    return ((uint64_t) from * (uint64_t) to) >> 32;
}

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}
'''

In [151]:
main_src, fuzz_src = CFuzzerPRNG(c_grammar).fuzz_src()
with open('testers/grammar_producer_cprng_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_cprng_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [152]:
!cat testers/grammar_producer_cprng_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>


int max_depth = 0;

void gen_init__();

uint64_t next(void);
uint32_t map(uint32_t to) {
    uint32_t from = next();
    return ((uint64_t) from * (uint64_t) to) >> 32;
}

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}


void out(const char s) {
    fputc(s, stdout);
}       
 

int main(int argc, char** argv) {
    int seed, max_num;

    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\n", argv[0

In [153]:
%cd testers
!cc -Ofast grammar_producer_cprng_main.c grammar_producer_cprng_fuzz.c -o grammar_producer_cprng
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [154]:
class CTesterPRNG(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_cprng {seed} {self.max_num} {max_depth} > {fn}"

In [155]:
!./testers/grammar_producer_cprng 0 10 10

-(((9.9)/0.4/8.8)/8*+9)
8498+-+2*397.445*9+-+0230+78/(-0.8-0.8*4.3)/-(7.5-1.3)*3.62
-29.5726-4/1320
+6172
(98.2)*6654
(732.391*-2.440--(3.9*5.6)/(2.3))/+71-++(--0.5*7.5/6.5/1.5+(0)-6.6)/7/+-+++7.7/3*-96.27*+-4.4-(79.85*+(2.4)/4/9.3*8+(8.3-1.4)*(7.2)/0*4.1++8.5*9.4-9*6.8-2.7)++63.6--8*762.9/(8)/9/2.0+-6.3*80.56*2.0*6.0/0-82.91*-9.7+6*7.1
2
708*1-4345-730/(5.53/1.6+6/5.0/1.2)*(-1.8*6.4/1.8+7.7-9.6)-((1.0-4.2)*(5.8)*0.6*0.7-2)++++9.6*177.4*51-+98+2.8*7-+1*3.8/3
+--(-+8.1*4.2--9.4-8.1*2+9.4-1.9)/1.57+8
-5*7886406.09/((1.4+8.9-6.7)-1.96*0/6.7)++0.07/-6


In [156]:
CTesterPRNG().run_test().show()

depth= 8 size= 57763 time= 0.005 stdev(0.003) throughput= 13430.75706845238 stdev(7598)
depth= 16 size= 255864 time= 0.013 stdev(0.004) throughput= 19165.949041193184 stdev(5019)
Throughput of  19165.949041193184  kilobytes per second at depth =  16
Total time: 0:00:08.899275


### Can we make the random go faster?

**idea**: Do the random allocation in one place, and use that later.

#### How fast is /dev/random (and variants)?

Using the best block size, and fastest #counts

In [157]:
with timeit() as t:
    !dd if=/dev/random of=random.x bs=1024 count=1000 2>/dev/null
print("throughput=",os.stat('random.x').st_size/1024/t.runtime, 'kb per second')

throughput= 62821.96255808539 kb per second


In [158]:
with timeit() as t:
    !dd if=/dev/urandom of=random.x bs=1024 count=1000 2>/dev/null
print("throughput=",os.stat('random.x').st_size/1024/t.runtime, 'kb per second')

throughput= 55478.50208045011 kb per second


In [159]:
with timeit() as t:
    !dd if=/dev/zero of=io.x bs=1024 count=1000 2>/dev/null
print("throughput=",os.stat('io.x').st_size/1024/t.runtime, 'kb per second')

throughput= 89823.04859426165 kb per second


**Idea**:
* Pre-allocate random bits, and use only as much as necessary.
* Optimize for < 256 bits by using only a single byte at a time.

```c
uint8_t
map(uint8_t to) {
    uint8_t from = rand_region[rand_cursor++];
    if (rand_cursor >= rand_region_size)
        rand_cursor = 0;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}
```

**idea**: Wraparound at 4 GB to avoid comparisons (this did not work as expected.)

In [160]:
u_max_int = 4096 * 1024 * 1024

(The better idea is to use a pointer to the last element, and increment it rather than use an array address, which is required for this trick.)

### Can we make random faster?

In [161]:
my_str = '''
#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>
#include <errno.h>
#include <string.h>

uint8_t* rand_region;
void* stack[INT_MAX/9]; /*OSX problems. /9 is the max size.*/

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}

uint8_t* rand_region_size = 0;

void
__attribute__((flatten))
initialize_random(uint64_t max_chars) {
    uint64_t* arr = (uint64_t*) rand_region;
    uint64_t i;
    for (i=0; i < max_chars/8; i++) { /*max_space/8 because we have 8 bytes*/
        arr[i] = next();
    }
    rand_region_size = (uint8_t*) (arr+i);
}

int main(int argc, char** argv) {
    struct stat st;
    int rand_fd;
    uint8_t* rand_region_init;

    char* rand_file = argv[1];
    rand_fd = open(rand_file, O_RDWR | O_CREAT, 0600);
    size_t u_max = (uint64_t)powl(2,32);
    int res = ftruncate(rand_fd, u_max);
    if (res == -1) {
        fprintf(stdout, "Error: %s\\n", strerror(errno));
        return 4;
    }
    rand_region = mmap(0, u_max, PROT_READ| PROT_WRITE, MAP_SHARED, rand_fd, 0);
    rand_region_init = rand_region;
    if (rand_region == (uint8_t*)-1) {
        exit(3);
    }
    initialize_random(u_max);
    msync(rand_region, st.st_size, MS_SYNC);
    munmap(rand_region, st.st_size);
    long rand_size = rand_region_size - rand_region_init;
    ftruncate(rand_fd, rand_size);
    /*fprintf(stdout, "%ld\\n", rand_size);*/
    close(rand_fd);
    return 0;
}
'''

In [162]:
with open('testers/rand.c', 'w+') as f:
    print(my_str, file=f)

In [163]:
%cd testers
!cc -g -o rand -lm -Ofast rand.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [164]:
with timeit() as t:
    !./testers/rand random1.x
print("throughput=",os.stat('random1.x').st_size/1024/t.runtime)
!rm random1.x

throughput= 699318.3890566103


Allocate a file `u_max_int` size, and mmap it to memory.

```c
uint32_t rand_cursor = 0;
uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = rand_region[rand_cursor++];
    return ((uint16_t) from * (uint16_t) to) >> 8;
}
```

Does it help?

In [165]:
# II
# https://lemire.me/blog/2016/06/27/a-fast-alternative-to-the-modulo-reduction/
class CFuzzerExtRand(CFuzzer):
    def main_hdefs(self):
        s = super().main_hdefs()
        return s + '''
#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>
'''
    
    def fn_map_def(self):
        return '''
uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = rand_regionp[rand_cursor++];
    if (rand_cursor >= rand_region_size)
        rand_cursor = 0;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}

uint8_t* rand_region_sizep = 0;

void
__attribute__((flatten))
initialize_random(uint64_t max_chars) {
    uint64_t* arr = (uint64_t*) rand_regionp;
    uint64_t i;
    for (i=0; i < max_chars/8; i++) { /*max_space/8 because we have 8 bytes*/
        arr[i] = next();
    }
    rand_region_sizep = (uint8_t*) (arr+i);
}
'''
    def main_rand_var_defs(self):
        return '''
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_regionp[rand_region_size];
uint64_t rand_cursor = 0;
'''
    
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_rand_var_defs()

    def fuzz_hdefs(self):
        s = super().fuzz_hdefs()
        return s + '''
#include <unistd.h>
#include <stdint.h>'''
    
    def fuzz_rand_var_defs(self):
        return '''
extern uint8_t* rand_regionp;
extern uint64_t rand_cursor;
extern uint64_t rand_region_size;
uint8_t map(uint8_t to);'''
 
    def fn_main_rand_frag(self):
        return '''\
    initialize_random(rand_region_size);
    rand_cursor = seed;
    '''
 
    def fn_main_def(self):
        return '''
int main(int argc, char** argv) {
    struct stat st;
    int max_num, seed, rand_fd, out_fd;
    long out_size;
%(input_frag)s
%(rand_frag)s
%(loop_frag)s
    return 0;
}''' % { 'input_frag': self.fn_main_input_frag(),
         'loop_frag': self.fn_main_loop_frag(),
         'rand_frag': self.fn_main_rand_frag()}

In [166]:
main_src, fuzz_src = CFuzzerExtRand(c_grammar).fuzz_src()
with open('testers/grammar_producer_cprngextr_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_cprngextr_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [167]:
!cat testers/grammar_producer_cprngextr_fuzz.c


#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <stdint.h>

void out(const char s);

extern uint8_t* rand_regionp;
extern uint64_t rand_cursor;
extern uint64_t rand_region_size;
uint8_t map(uint8_t to);

extern int max_depth;
void gen_start(int depth);
void gen_expr(int depth);
void gen_term(int depth);
void gen_factor(int depth);
void gen_integer(int depth);
void gen_digit(int depth);

const char* pool_start[] =  {"1.8", "6.4", "6", "0", "9.8", "7.1", "6.1", "6.5", "1.3", "8.0", "3.4", "4.2", "0.0", "4.3", "3.9", "9.5", "7.3", "5.9", "6.0", "1.6", "1", "1.5", "2.5", "4.9", "9.3", "0.8", "3.3", "8", "7.8", "6.8", "7.0", "5.8", "5.2", "2.6", "9", "6.3", "5.6", "9.9", "4.1", "5.3", "0.3", "2.2", "6.7", "3", "9.7", "8.7", "0.5", "9.1", "5", "7.6", "8.5", "8.1", "8.9", "2.0", "2.1", "8.8", "9.6", "8.6", "0.6", "4.0", "1.7", "5.4", "1.1", "3.2", "7.7", "3.7", "9.0", "8.3", "1.0", "1.2", "4", "3.0", "5.5", "9.4", "1.9", "2.8", "7.9"

In [168]:
!nl -ba testers/grammar_producer_cprngextr_main.c

     1	
     2	#define _LARGEFILE64_SOURCE
     3	#define _FILE_OFFSET_BITS 64
     4	
     5	#include <stdlib.h>
     6	#include <stdio.h>
     7	#include <time.h>
     8	#include <string.h>
     9	
    10	#include <unistd.h>
    11	#include <limits.h>
    12	#include <fcntl.h>
    13	#include <sys/types.h>
    14	#include <sys/mman.h>
    15	#include <sys/stat.h>
    16	#include <math.h>
    17	
    18	
    19	int max_depth = 0;
    20	
    21	void gen_init__();
    22	const uint64_t rand_region_size = 1ULL << 16;
    23	uint8_t rand_regionp[rand_region_size];
    24	uint64_t rand_cursor = 0;
    25	
    26	
    27	uint8_t
    28	__attribute__((always_inline))
    29	map(uint8_t to) {
    30	    uint8_t from = rand_regionp[rand_cursor++];
    31	    if (rand_cursor >= rand_region_size)
    32	        rand_cursor = 0;
    33	    return ((uint16_t) from * (uint16_t) to) >> 8;
    34	}
    35	
    36	static inline uint64_t rotl(const uint64_t x, int k) {
    37	    return (x << k) | (x 

In [169]:
%cd testers
!cc -g -Ofast -o grammar_producer_cprngextr grammar_producer_cprngextr_main.c grammar_producer_cprngextr_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [170]:
# II
!./testers/grammar_producer_cprngextr 0 10 10

0/2
0
30.3/-2.042918/0417.5/-+8.5---(-+7.6/3.0)*--(+2.2/7.0-3.3*4-4.5+2.0)*((7.8-6.2))*9.30*-++8.9--765*2/(9.4-0.5*3.2-1.8+0)-1.3855
2
++85.5*(9.58*+8.5+(1.2*4.3)/-(7.6)-+5-+5.0*0.3+2.6+5.7-1.0)
+(2141.7574*6.19+--7.5/3*-4*0.1*6.0)*4/0*(+(6)/(3.5))/-831++--6*(4-+-0/3.7+(8.9)/0.9*3.5+3)*1.27/840/91.7*+3/+0.1*6+-2*1*8146.91/7*+7.9/-4.4-15.2/7*4/(5.4+7.3)*-6.2*9.4*9.4-(+9.9+0/5.2)*7+985.9
3.342+(19+0.93*6/+5.7/7+-2-(0.8)/2.8*0.8)
-3/+912*-6++--2.3098/((0.9+6.9+3.6)/39.14*(2.6)/1.4/2.5)/70*-757/-2.9++((3.1)+(9.3)/8.6-1.0/4.1-0-9.9)+--44.4*+9*(5.0+1.0+7.5)-(+7.8*0*5.1)/+5/(7.9+6.3)*2.9-0-4.53*-2.2/5.5*0.8
++966++974.1
00.010/-+++-(5+7.5)


In [171]:
# II
class CTesterPRNGExt(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_cprngextr {seed} {self.max_num} {max_depth} ./random.x > {fn}"

In [172]:
CTesterPRNGExt().run_test().show()

depth= 8 size= 60087 time= 0.007 stdev(0.001) throughput= 8556.498209635418 stdev(1721)
depth= 16 size= 251558 time= 0.015 stdev(0.001) throughput= 15865.346272786459 stdev(709)
Throughput of  15865.346272786459  kilobytes per second at depth =  16
Total time: 0:00:10.018077


In [173]:
# II
class CFuzzerExtRandP(CFuzzerExtRand):
    def fn_map_def(self):
        return '''
uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}

void
__attribute__((flatten))
initialize_random(uint64_t max_chars) {
    uint64_t* arr = (uint64_t*) rand_regionp;
    uint64_t i;
    for (i=0; i < max_chars/8; i++) { /*max_space/8 because we have 8 bytes*/
        arr[i] = next();
    }
    rand_region_sizep = (uint8_t*) (arr+i);
}
'''
    def main_rand_var_defs(self):
        return '''
uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;
'''
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);
'''
    
    def fn_main_loop_frag(self):
        return '''
    for (int i = 0; i < max_num; i++) {
        gen_init__();
    }
'''
    def fn_main_rand_frag(self):
        return '''\
    initialize_random(rand_region_size);
    rand_regionp += seed;
    '''
    def fn_main_def(self):
        return '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_sizep = 0;
    char* out_region_initp;
    int out_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(loop_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag()
       }

In [174]:
main_src, fuzz_src = CFuzzerExtRandP(c_grammar).fuzz_src()
with open('testers/grammar_producer_cprngextrP_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_cprngextrP_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [175]:
!cat testers/grammar_producer_cprngextrP_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>


int max_depth = 0;

void gen_init__();
uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;


uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

  

In [176]:
!cat testers/grammar_producer_cprngextrP_fuzz.c


#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <stdint.h>

void out(const char s);

uint8_t map(uint8_t to);


extern int max_depth;
void gen_start(int depth);
void gen_expr(int depth);
void gen_term(int depth);
void gen_factor(int depth);
void gen_integer(int depth);
void gen_digit(int depth);

const char* pool_start[] =  {"5.9", "7.2", "2.6", "3.4", "9.0", "1.3", "6.9", "4.2", "8.7", "1.0", "1.6", "5.6", "7.6", "7.3", "5.0", "7.8", "7.4", "7.9", "6.6", "1.4", "4.4", "8.0", "3.6", "6", "2.0", "5.5", "2.4", "5.8", "3.3", "2", "8.2", "4.0", "1.8", "5.1", "1.7", "4.3", "5.3", "6.7", "1.5", "8.6", "9.6", "3.7", "0.3", "7.0", "5.2", "8.3", "8.4", "3.8", "5.7", "7", "4.5", "2.9", "2.5", "9.1", "8", "4.7", "5", "9.8", "2.3", "8.5", "4.8", "9.7", "9", "0.6", "7.1", "3.5", "0.7", "4.9", "3", "0.8", "9.3", "5.4", "3.0", "0.0", "2.1", "8.9", "9.4", "1.2", "9.9", "6.3", "0.9", "3.9", "1", "8.1", "2.7", "6.8", "3.2", "6.2", "0.2", "9.5",

In [177]:
%cd testers
!cc -g -Ofast -o grammar_producer_cprngextrP grammar_producer_cprngextrP_main.c grammar_producer_cprngextrP_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [178]:
!./testers/grammar_producer_cprngextrP 0 10 10

0/2
0
30.3/-2.042991/0417.5/-+8.5---(-+9.9/9.6)*--(+4.2/6.1-4.3*0-6.5+7.3)*((3.6-1))*9.30*-++7.8--765*2/(7.8-0.2*9.6-2.7+0)-1.3858
2
++85.5*(9.58*+4.8+(3.4*2.3)/-(4.2)-+5-+3.5*4.4+7.4+2.9-8.3)
+(2152.7568*6.13+--4.5/7*-0.8*5.9*5.3)*4/0*(+(5.1)/(3))/-872++--6*(4-+-6.3/9.4+(2.3)/0.9*1.4+2.1)*1.27/841/99.7*+9/+5.9*7.3+-2*1*8150.91/7*+4.7/-9.1-15.2/7*4/(0.0+3.7)*-1.8*4.9*7.6-(+9.5+6.3/1.9)*7+945.9
3.342+(19+0.97*2/+6.8/9.6+-3-(1.6)/5.5*3.8)
-3/+912*-6++--2.3031/((0.9+0.2+8.0)/77.98*(9.3)/2.3/8.7)/70*-784/-0.3++((8.1)+(5.9)/8.9-0.3/6.2-8.5-6.2)+--45.4*+9*(3.5+2.4+3.2)-(+8.0*6.3*4.9)/+8/(6.4+4.3)*3.7-0-5.89*-4.2/7.5*3.8
++966++974.1
00.010/-+++-(3.3+3.2)


In [179]:
# II
class CTesterPRNGExtP(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_cprngextrP {seed} {self.max_num} {max_depth} > {fn}"

In [180]:
CTesterPRNGExtP().run_test().show()

depth= 8 size= 59976 time= 0.007 stdev(0.002) throughput= 9516.24755859375 stdev(3097)
depth= 16 size= 251066 time= 0.015 stdev(0.001) throughput= 15834.315999348959 stdev(708)
Throughput of  15834.315999348959  kilobytes per second at depth =  16
Total time: 0:00:09.569741


## Using faster IO.

### MMap

**Idea**:
* `mmap` to a file, write the bits, and `ftruncate()` to the new size.

In [181]:
# II
class CMMapIOFuzzer(CFuzzerExtRandP):
    def main_out_var_defs(self):
        return '''
char* out_regionp;
uint64_t out_cursor = 0;
'''
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_out_var_defs()
     
    def fn_out_def(self):
        return '''
void
__attribute__((always_inline))
out(char c) {
    out_regionp[out_cursor++] = c;
}
'''
    
    def fuzz_out_var_defs(self):
        return '''
void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;
'''

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''

    def fn_main_out_frag(self):
        return '''
    char* iomax = getenv("IO_LIMIT");
    uint64_t u_iomax = UINT_MAX * 10ULL; // 40G
    if (iomax) {
        u_iomax = 1ULL << atoi(iomax);
    }
    if (argc > 4) {
        out_fd = open(argv[4], O_RDWR | O_CREAT, 0600);
    } else {
        out_fd = open("io.x", O_RDWR | O_CREAT, 0600);
    }
    if (iomax) {
        int res = ftruncate(out_fd, u_iomax);
        if (res != 0) {
            perror("truncate failed");
            exit(2);
        }
    } else {
        int res = try_truncate(out_fd);
        if (res < 32) {
            perror("truncate failed");
            fprintf(stderr,"%d\\n", res);
            exit(5);
        }
    }
    fstat(out_fd, &st);
    out_regionp = mmap(0, st.st_size, PROT_READ|
                      PROT_WRITE, MAP_SHARED, out_fd, 0);
    if (out_regionp == (caddr_t)-1) {
        exit(3);
    }
    '''
    
    def fn_main_sync_frag(self):
        return '''
    msync(out_regionp, st.st_size, MS_SYNC);
    munmap(out_regionp, st.st_size);
    ftruncate(out_fd, out_cursor);
    close(out_fd);
'''

    def fn_truncateio(self):
        return '''
#include <errno.h>
int try_truncate(int fd) {
    for (off_t len = 63; len > 0; len--) {
      uint64_t m = 1ULL << len;
      errno = 0;
      int ret = ftruncate(fd, m);
      if (ret == 0) {
        return len;
      }
    }
    return -1;
}
'''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd, out_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'out_frag': self.fn_main_out_frag(),
        'sync_frag': self.fn_main_sync_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag(),
       }

In [182]:
main_src, fuzz_src = CMMapIOFuzzer(c_grammar).fuzz_src()
with open('./testers/grammar_producer_mmapio_main.c', 'w+') as f:
    print(main_src, file=f)
with open('./testers/grammar_producer_mmapio_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [183]:
!cat testers/grammar_producer_mmapio_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>


int max_depth = 0;

void gen_init__();
uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;

char* out_regionp;
uint64_t out_cursor = 0;


uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t re

In [184]:
%cd testers
!cc -g -Ofast -o grammar_producer_mmapio grammar_producer_mmapio_main.c grammar_producer_mmapio_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [185]:
# II
!./testers/grammar_producer_mmapio 0 10 10 io.x

In [186]:
!du -ksh io.x

1.0M	io.x


In [187]:
# II
class CTesterMMap(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_mmapio {seed} {self.max_num} {max_depth} {fn}"

In [188]:
CTesterMMap().run_test().show()

KeyboardInterrupt: 

### FWrite

In [189]:
class CFWriteFuzzer(CFuzzerExtRandP):
    def main_out_var_defs(self):
        return '''
const uint64_t size = UINT_MAX; /*max size of a single input -- 4G*/
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;
FILE* fs;
'''
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_out_var_defs()
     
    def fn_out_def(self):
        return '''
void
__attribute__((always_inline))
out(char c) {
    out_regionp[out_cursor++] = c;
}'''
    
    def fuzz_out_var_defs(self):
        return '''
void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;
'''

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''

    def fn_main_out_frag(self):
        return '''
    if (argc > 4) {
        out_fd = open(argv[4], O_RDWR | O_CREAT, 0600);
    } else {
        out_fd = open("io.x", O_RDWR | O_CREAT, 0600);
    }
    fs = fdopen(out_fd, "w");
'''

    def fn_main_sync_frag(self):
        return '''
    fclose(fs);
    close(out_fd);
'''

    def fn_truncateio(self):
        return '''
'''
    def fn_main_loop_frag(self):
        return '''
    for(int i=0; i < max_num; i++) {
        gen_init__();
        fwrite(out_regionp, sizeof(char), out_cursor, fs);
        out_cursor = 0;
    }
'''

    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd, out_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'out_frag': self.fn_main_out_frag(),
        'sync_frag': self.fn_main_sync_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag(),
       }

In [190]:
main_src, fuzz_src = CFWriteFuzzer(c_grammar).fuzz_src()
with open('./testers/grammar_producer_fwrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('./testers/grammar_producer_fwrite_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [191]:
!cat testers/grammar_producer_fwrite_fuzz.c


#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <stdint.h>

void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;


uint8_t map(uint8_t to);


extern int max_depth;
void gen_start(int depth);
void gen_expr(int depth);
void gen_term(int depth);
void gen_factor(int depth);
void gen_integer(int depth);
void gen_digit(int depth);

const char* pool_start[] =  {"6.4", "6.6", "3.4", "6", "8.6", "5.2", "4.2", "1.5", "7.7", "2.8", "7.2", "3.3", "2.1", "6.1", "5.1", "8.5", "6.3", "6.9", "4.4", "8", "0.9", "2.9", "2.4", "1.0", "5.3", "9.8", "0.3", "8.7", "5.8", "4.9", "4", "0.6", "7.4", "4.5", "6.2", "6.7", "3.1", "2", "5.9", "9", "2.3", "9.4", "9.2", "1.6", "1.3", "3.8", "0.1", "6.8", "9.9", "2.6", "9.6", "2.0", "9.3", "6.0", "3", "7", "8.0", "4.1", "8.3", "4.8", "5.4", "0.7", "6.5", "0", "7.0", "1.7", "1.2", "1.8", "3.9", "0.8", "0.2", "9.1", "3.7", "8.4", "3.5", "5.0", "8.1", "1.4", "4.6", "4.0", "7.8", "4.7", "0.5",

In [192]:
!cat testers/grammar_producer_fwrite_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>


int max_depth = 0;

void gen_init__();
uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;

const uint64_t size = UINT_MAX; /*max size of a single input -- 4G*/
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;
FILE* fs;


uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static u

In [193]:
%cd testers
!cc -g -Ofast -o grammar_producer_fwrite grammar_producer_fwrite_main.c grammar_producer_fwrite_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [194]:
# II
!./testers/grammar_producer_fwrite 0 10 10 io.x

dyld[38747]: dyld cache '(null)' not loaded: syscall to map cache into shared region failed
dyld[38747]: Library not loaded: /usr/lib/libSystem.B.dylib
  Referenced from: <087EBBBB-08CA-3C0E-AC07-9D5BF1AD091F> /Users/rahul.gopinath/Research/f1/F1/testers/grammar_producer_fwrite
  Reason: tried: '/usr/lib/libSystem.B.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/lib/libSystem.B.dylib' (no such file), '/usr/lib/libSystem.B.dylib' (no such file, no dyld cache)


In [ ]:
!du -ksh io.x

In [195]:
# II
class CTesterFWrite(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_fwrite {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterFWrite().run_test().show()

### No output 

In [196]:
class CNoWriteFuzzer(CFuzzerExtRandP):
    def main_out_var_defs(self):
        return '''
const uint64_t size = UINT_MAX; // size of a single output item -- 4G
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;'''
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_out_var_defs()
     
    def fn_out_def(self):
        return '''
void
__attribute__((always_inline))
out(char c) {
    out_regionp[out_cursor++] = c;
}'''
    
    def fuzz_out_var_defs(self):
        return '''\
void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;
'''

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''

    def fn_main_out_frag(self):
        return '''
    '''
    
    def fn_main_sync_frag(self):
        return '''
    '''

    def fn_truncateio(self):
        return '''
        '''
    def fn_main_loop_frag(self):
        return '''
    uint64_t out_size = 0;
    for(int i=0; i < max_num; i++) {
        gen_init__();
        // throw away
        out_size += out_cursor;
        out_cursor = 0;
    }
    printf("%lld\\n", out_size);
    '''

    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'out_frag': self.fn_main_out_frag(),
        'sync_frag': self.fn_main_sync_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag(),
       }

In [197]:
main_src, fuzz_src = CNoWriteFuzzer(c_grammar).fuzz_src()
with open('./testers/grammar_producer_nowrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('./testers/grammar_producer_nowrite_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [198]:
%cd testers
!cc -g -Ofast -o grammar_producer_nowrite grammar_producer_nowrite_main.c grammar_producer_nowrite_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [199]:
# II
!./testers/grammar_producer_nowrite 0 10 10

dyld[38769]: dyld cache '(null)' not loaded: syscall to map cache into shared region failed
dyld[38769]: Library not loaded: /usr/lib/libSystem.B.dylib
  Referenced from: <7C8D1978-8A0C-35D8-90CD-2B299B7DEB83> /Users/rahul.gopinath/Research/f1/F1/testers/grammar_producer_nowrite
  Reason: tried: '/usr/lib/libSystem.B.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/lib/libSystem.B.dylib' (no such file), '/usr/lib/libSystem.B.dylib' (no such file, no dyld cache)


In [200]:
# II
class CTesterNoWrite(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_nowrite {seed} {self.max_num} {max_depth} > {fn}"
  
    def post_time(self):
        super().post_time()
        with open(self.file) as f:
            self.size = int(f.read())

In [ ]:
CTesterNoWrite().run_test().show()

## Fuzzer as a VM

### MMap

#### Direct threaded VM

In [202]:
# II
class DTMMapFuzzer(CMMapIOFuzzer):
    def fn_out_def(self): return ''
    def gen_rule_src(self, rule, k, j):
        res = []
        leaf = True
        for i, token in enumerate(rule):
            if token in self.grammar:
                leaf = False
                trules = self.grammar[token] # ordered by cost
                len_min_choices = len(self.c_grammar[token])
                assert len(trules) < 256
                cheap_strings = self.pool_of_strings[token]
                if len(cheap_strings) < 256: # we only have 255 random choices
                    check_pool = '''
        val = map(%(len_cheap_strings)s);
        const char* str = pool_%(k)s[val];
        const int str_l = pool_l_%(k)s[val];
        for (int i = 0; i < str_l; i++) {
            *out_regionp++ = str[i];
        }
        --returnp;
        goto **returnp; 
            ''' % { 'len_cheap_strings': len(cheap_strings), 'k': self.k_to_s(token)}
                else:
                    check_pool = '''
        val = map(%(len_min_choices)s);
                ''' % {'len_min_choices':len_min_choices}
                res.append('''\
    *returnp = &&return__%(i)d__%(j)d__%(k)s;
    if (returnp > max_depthp) {
        %(check_pool)s;
    } else {
        val = map(%(len_rules)s);
    }
    goto *gen_%(t)s[val];
return__%(i)d__%(j)d__%(k)s:;
            ''' % {'i':i, 'j':j, 'k':self.k_to_s(k),
                   't':self.k_to_s(token), 'rnum':0, 'len_rules':len(trules), 'len_min_choices':len_min_choices, 'check_pool':check_pool})
            else:
                res.append('''\
    *out_regionp++ = '%s';''' % self.esc_char(token))
        return res, leaf
    
    def gen_alt_src_1rule(self, k):
        rule = self.grammar[k][0]
        ri = 0
        src, leaf = self.gen_rule_src(rule, k, ri)
        body = '\n'.join(src)
        result = []
        if leaf:
            return '''
gen_%(name)s_0: {
%(body)s
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}
        else:
             return '''
gen_%(name)s_0: {
    ++returnp;
    // single -- no switch
%(body)s
    --returnp;
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        ret = self.k_to_s(k)
        result = []
        if len(rules) == 1: return self.gen_alt_src_1rule(k)
        for ri, rule in enumerate(rules):
            src, leaf = self.gen_rule_src(rule, k, ri)
            body = '\n'.join(src)
            if leaf:
                result.append('''
gen_%(name)s_%(rnum)d: {
%(body)s
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
            else:
                 result.append('''
gen_%(name)s_%(rnum)d: {
    ++returnp;
%(body)s
    --returnp;
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
        return '\n'.join(result)

    def fuzz_out_var_defs(self):
        return '''\
extern char* out_regionp;'''
    
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);'''
    
    def fuzz_stack_var_defs(self):
        return '''
extern void* stackp[];
'''

    def fuzz_entry(self):
        result = ['''
void gen_init__(void** max_depthp) {
    uint8_t val;
    void** returnp = stackp;
    *returnp =  &&return__init;
''']
        for k in self.grammar:
            l = []
            for ri,rule in enumerate(self.grammar[k]):
                l.append('&&gen_%(k)s_%(ri)d' % {'k':self.k_to_s(k), 'ri':ri})
            s = '''
    void** gen_%(k)s[] = {
%(body)s
    };''' % {'k': self.k_to_s(k), 'body': ',\n'.join(l)}
            result.append(s)
        result.append('''
    goto gen_start_0;''')
        result.append(self.fuzz_fn_defs())
        result.append("""
return__init:
    return;
return_abort:
    exit(10); 
}""")
        return '\n'.join(result)
    
    def main_out_var_defs(self):
        return'''
char* out_regionp;
int out_cursor;
'''
    
    def main_stack_var_defs(self):
        return'''
int max_depth;
void** max_depthp;
void* stackp[INT_MAX];
'''
    def main_init_var_defs(self):
        return'''
void gen_init__(void** max_depthp);
'''

    def fn_main_loop_frag(self):
        return '''
    for(int i=0; i < max_num; i++) {
        gen_init__(max_depthp);
        *out_regionp++ = '\\n';
    }
    *out_regionp = 0;'''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_sizep = 0;
    char* out_region_initp;
    int rand_fd, out_fd;
    int seed, max_num;
%(input_frag)s
    max_depthp = stackp + max_depth;
%(rand_frag)s
%(out_frag)s
    out_region_initp = out_regionp;
    out_region_sizep = out_regionp + st.st_size;
%(loop_frag)s
    out_size = out_regionp - out_region_initp;
    out_cursor = out_size;
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }


    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          # self.fuzz_fn_defs(),
                          self.fuzz_entry()])

In [203]:
main_src, fuzz_src = DTMMapFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_dtmmap_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_dtmmap_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [204]:
!wc -l testers/grammar_producer_dtmmap_fuzz.c

     627 testers/grammar_producer_dtmmap_fuzz.c


In [205]:
%cd testers
!cc -g -Ofast -o grammar_producer_dtmmap grammar_producer_dtmmap_main.c grammar_producer_dtmmap_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [206]:
# II
!./testers/grammar_producer_dtmmap 0 10 10 io.x

dyld[38793]: dyld cache '(null)' not loaded: syscall to map cache into shared region failed
dyld[38793]: Library not loaded: /usr/lib/libSystem.B.dylib
  Referenced from: <A968B940-6DE0-330D-81F0-BF328D9AD0D9> /Users/rahul.gopinath/Research/f1/F1/testers/grammar_producer_dtmmap
  Reason: tried: '/usr/lib/libSystem.B.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/lib/libSystem.B.dylib' (no such file), '/usr/lib/libSystem.B.dylib' (no such file, no dyld cache)


In [ ]:
!du -ksh io.x

In [207]:
# II
class CTesterMMapDT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_dtmmap {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterMMapDT().run_test().show()

#### Context threaded VM

In [208]:
# II
class CTMMapFuzzer(DTMMapFuzzer):
    def fn_choice(self, val):
        return '''
    # [ random 
    # extract one byte from the random stream %%r14,
    movq (%%r14), %%rdi
    # advance the random cursor
    inc %%r14                                     # rand_region++
    movzbl %%dil, %%edi                           # X  --- (rdi:(edi:(di:(dil))))
    # then multiply with the choices we have

    xor %%rsi, %%rsi                              # avoid data dependencies
    movb $%(val)s, %%sil                          # = %(val)s       
    movzbl %%sil, %%edx
    imull %%edi, %%edx                            # m = (short) x * (short) N)
    sarl $8, %%edx                                # return (char)(m >> 8) ;
    # random ]
    # %%edx now contains the selected random value from %(val)d options''' % {'val':val}

    def cheap_strings(self, k):
        cheap_strings = self.pool_of_strings[k]
        results = ['''
    # --- cheap -- [''']
        results.append('''
%(choices)s
''' % {'choices':self.fn_choice(len(cheap_strings)), 'len_choices': len(cheap_strings)})
        # get the choices from vm, then call it, and return.
        
        results.append('''
    # now we have the right print quad in %%edx. Load the right address and call it.
    leaq _%(key)s_prints(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ret
    ''' % {'key': self.k_to_s(k)})
        results.append('''
    # --- cheap -- ]''')
        return '\n'.join(results)
    
    def output_char(self, c):
        if len(c) != 1:
            assert c[0] == '\\'
            c = c[-1]
        return '''
   movb $%(ichar)d, (%%r13)                     # '%(char)s'
   inc %%r13                                    # out_region++   : increment a byte (r13++)
   ''' % {'char':repr(c), 'ichar':ord(c)}

    def gen_rule_src(self, rule, k, j):
        # in each rule, there are a number of tokens.
        # iter each token in turn, choose the right rule and call.
        result = []
        for token in rule:
            if token not in self.grammar:
                result.append(self.output_char(token))
                continue
            else:
                # how many choices do we have?
                rules = self.grammar[token]
                result.append('''
    # start the choice machine.
    # length of rules = %(len_rules)d
%(choices)s
    # --- switch ---
    ''' % {'choices': self.fn_choice(len(rules)), 'len_rules':len(rules)})
                result.append('''
    # now we have the right choice in %%edx. Load the right address and call it.
    leaq _%(key)s_choices(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ''' % {'key': self.k_to_s(token)})
        return '\n'.join(result)

    def gen_alt_src(self, k):
        result = []
        for ruleid, rule in enumerate(self.grammar[k]):
            # produce a skeletal subroutine structure.
            result.append('''
gen_%(key)s_%(ruleid)s:
    # check if the max depth is breached.
    cmpq %%rsp, %%r8                             # returnp(rbp) <> max_depth(r8) ?
    jle _%(key)s_%(ruleid)s_fi                       # returnp <= max_depth
    
%(return_cheap_string)s
_%(key)s_%(ruleid)s_fi:
''' % {'return_cheap_string': self.cheap_strings(k),
       'key':self.k_to_s(k),
       'ruleid':ruleid,
       'last_label':self.last_label})
            self.last_label += 1
            result.append(self.gen_rule_src(rule, k, ruleid))
            # we were called. So simply return.
            result.append('''
    ret
            ''')
        return '\n'.join(result)
 
    def fn_fuzz_decs(self):
        result = ['''
  .section  __DATA,__data

# Virtual Machine OPS.
        ''']
        for k in self.grammar:
            result.append('''
    .globl  _%(key)s_choices
    .p2align 4
_%(key)s_choices:''' % {'key':self.k_to_s(k)})
            for i, rule in enumerate(self.grammar[k]):
                result.append('''\
    .quad gen_%s_%d''' % (self.k_to_s(k), i))
                
        for k in self.pool_of_strings:
            result.append('''
    .globl  _%(key)s_prints
    .p2align 4
_%(key)s_prints:''' % {'key':self.k_to_s(k)})
            for string in self.pool_of_strings[k]:
                result.append('''\
    .quad %s''' % (self.all_prints[string]))
                
                
        result.append('''
# End Virtual Machine OPS.''')
        return '\n'.join(result)

    def gen_cheap(self, grammar):
        all_strings = set()
        for k in grammar:
            all_strings |= set(self.pool_of_strings[k])
        all_strings = list(all_strings)
        all_strings.sort(key=lambda item: (-len(item), item))
        all_prints_hash = {}
        result = ['''
.text
        ''']
        for i, s_ in enumerate(all_strings):
            s = s_
            result.append('''\
print_%(name)d: # "%(value)s"''' % {'name': i, 'value': repr(s)})
            for j in s:
                result.append('''\
    movb $%(ichar)s, (%%r13)            # '%(char)s'
    inc %%r13''' % {'ichar':ord(j), 'char':repr(j)})
            result.append('''\
    ret''')
            all_prints_hash[s_] = 'print_%d' % i
        return ('\n'.join(result), all_prints_hash)
 
    def fuzz_entry(self):
        result = ["""
#include "ctmmap_vm_ops.s"
.macro pushaq
    push %%rsp
    push %%rbp
    push %%r8
    push %%r9
    push %%r10
    push %%r11
    push %%r12
    push %%r13
    push %%r14
    push %%r15
.endm


.macro popaq
    pop %%r15
    pop %%r14
    pop %%r13
    pop %%r12
    pop %%r11
    pop %%r10
    pop %%r9
    pop %%r8
    pop %%rbp
    pop %%rsp
.endm

.global %(os)sgen_init__
.global return__init
.text
%(os)sgen_init__:
    # 1 rdi = max_depth
    # 2 rsi = returnp
    # 3 rdx = &out_region
    # 4 rcx = &rand_region
    pushaq

    leal 0(,%%rdi,8), %%eax
    movq %%rsp, %%r8
    subq %%rax, %%r8

    movq %%rdx, %%r11                              # &out_region
    movq %%rcx, %%r12                              # &rand_region
    movq (%%r11),%%r13                             # out_region
    movq (%%r12),%%r14                             # rand_region

    # general regs
    # rax, rcx, rdx, rbx, rsi,rdi
    # rbp, r8-r15
    
    call gen_start_0
    movq %%r13, (%%r11)                            # *(&out_region) <-
    movq %%r14, (%%r12)                            # *(&rand_region) <-
    popaq
    movq  $0, %%rax
    ret   
""" % {'os': '_' if sys.platform == 'darwin' else ''}]
        result.append(self.fuzz_fn_defs())
        return ''.join(result)

    def main_init_var_defs(self):
        return'''\
void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
'''

    def fn_main_loop_frag(self):
        return '''\
    for(int i=0; i < max_num; i++) {
        gen_init__(max_depth32, stackp, &out_regionp, &rand_regionp);
        *out_regionp++ = '\\n';
    }
    *out_regionp = 0;'''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_initp;
    int out_fd;
    uint32_t max_depth32;
    int seed, max_num;
%(input_frag)s
    max_depth32 = max_depth;
%(rand_frag)s
%(out_frag)s
    out_region_initp = out_regionp;
%(loop_frag)s
    out_size = out_regionp - out_region_initp;
    out_cursor = out_size;
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }
    
    def fuzz_src(self, key='<start>'):
        self.last_label = 0
        self.cheap, self.all_prints = self.gen_cheap(self.grammar)
        ext_strings = '\n'.join([self.fn_fuzz_decs(), self.cheap])
        return ext_strings, self.gen_main_src(), self.gen_fuzz_src()
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_entry()])

In [209]:
vm_ops, main_src, fuzz_src = CTMMapFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_ctmmap_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_ctmmap_fuzz.s', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/ctmmap_vm_ops.s', 'w+') as f:
    print(vm_ops, file=f)

In [210]:
!nl -ba testers/grammar_producer_ctmmap_fuzz.s

     1	
     2	#include "ctmmap_vm_ops.s"
     3	.macro pushaq
     4	    push %rsp
     5	    push %rbp
     6	    push %r8
     7	    push %r9
     8	    push %r10
     9	    push %r11
    10	    push %r12
    11	    push %r13
    12	    push %r14
    13	    push %r15
    14	.endm
    15	
    16	
    17	.macro popaq
    18	    pop %r15
    19	    pop %r14
    20	    pop %r13
    21	    pop %r12
    22	    pop %r11
    23	    pop %r10
    24	    pop %r9
    25	    pop %r8
    26	    pop %rbp
    27	    pop %rsp
    28	.endm
    29	
    30	.global _gen_init__
    31	.global return__init
    32	.text
    33	_gen_init__:
    34	    # 1 rdi = max_depth
    35	    # 2 rsi = returnp
    36	    # 3 rdx = &out_region
    37	    # 4 rcx = &rand_region
    38	    pushaq
    39	
    40	    leal 0(,%rdi,8), %eax
    41	    movq %rsp, %r8
    42	    subq %rax, %r8
    43	
    44	    movq %rdx, %r11                              # &out_region
    45	    movq %rcx, %r12                              #

In [211]:
!nl -ba testers/grammar_producer_ctmmap_main.c

     1	
     2	#define _LARGEFILE64_SOURCE
     3	#define _FILE_OFFSET_BITS 64
     4	
     5	#include <stdlib.h>
     6	#include <stdio.h>
     7	#include <time.h>
     8	#include <string.h>
     9	
    10	#include <unistd.h>
    11	#include <limits.h>
    12	#include <fcntl.h>
    13	#include <sys/types.h>
    14	#include <sys/mman.h>
    15	#include <sys/stat.h>
    16	#include <math.h>
    17	
    18	
    19	int max_depth;
    20	void** max_depthp;
    21	void* stackp[INT_MAX];
    22	
    23	void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
    24	
    25	uint8_t* rand_region_sizep = 0;
    26	const uint64_t rand_region_size = 1ULL << 16;
    27	uint8_t rand_region_initp[rand_region_size];
    28	
    29	uint8_t* rand_regionp = rand_region_initp;
    30	
    31	char* out_regionp;
    32	int out_cursor;
    33	
    34	
    35	uint8_t
    36	__attribute__((always_inline))
    37	map(uint8_t to) {
    38	    uint8_t from = *rand_regionp++;

In [212]:
%cd testers
!cc -g -Ofast -o grammar_producer_ctmmap grammar_producer_ctmmap_main.c grammar_producer_ctmmap_fuzz.s
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
./ctmmap_vm_ops.s:544:16: error: unknown token in expression
    movb $48, (%r13) # ''0''
               ^
./ctmmap_vm_ops.s:544:15: error: invalid operand
    movb $48, (%r13) # ''0''
              ^
./ctmmap_vm_ops.s:546:16: error: unknown token in expression
    movb $46, (%r13) # ''.''
               ^
./ctmmap_vm_ops.s:546:15: error: invalid operand
    movb $46, (%r13) # ''.''
              ^
./ctmmap_vm_ops.s:548:16: error: unknown token in expression
    movb $48, (%r13) # ''0''
               ^
./ctmmap_vm_ops.s:548:15: error: invalid operand
    movb $48, (%r13) # ''0''
              ^
./ctmmap_vm_ops.s:552:16: error: unknown token in expression
    movb $48, (%r13) # ''0''
               ^
./ctmmap_vm_ops.s:552:15: error: invalid operand
    movb $48, (%r13) # ''0''
              ^
./ctmmap_vm_ops.s:554:16: error: unknown token in expression
    movb $46, (%r13) # ''.''
               ^
./ctmmap_vm_ops.s:554:15: error: invalid ope

In [ ]:
# II
!./testers/grammar_producer_ctmmap 0 10 10 io.x

In [ ]:
!du -ksh io.x

In [213]:
# II
class CTesterMMapCT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_ctmmap {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterMMapCT().run_test().show()

### FWrite

#### Direct threaded VM

In [214]:
# II
class CFWriteDTFuzzer(CFWriteFuzzer):
    def fn_out_def(self): return ''
    def gen_rule_src(self, rule, k, j):
        res = []
        leaf = True
        for i, token in enumerate(rule):
            if token in self.grammar:
                leaf = False
                trules = self.grammar[token] # ordered by cost
                len_min_choices = len(self.c_grammar[token])
                assert len(trules) < 256
                cheap_strings = self.pool_of_strings[token]
                if len(cheap_strings) < 256: # we only have 255 random choices
                    check_pool = '''
        val = map(%(len_cheap_strings)s);
        const char* str = pool_%(k)s[val];
        const int str_l = pool_l_%(k)s[val];
        for (int i = 0; i < str_l; i++) {
            *out_regionp++ = str[i];
        }
        --returnp;
        goto **returnp; 
            ''' % { 'len_cheap_strings': len(cheap_strings), 'k': self.k_to_s(token)}
                else:
                    check_pool = '''
        val = map(%(len_min_choices)s);
                ''' % {'len_min_choices':len_min_choices}
                res.append('''\
    *returnp = &&return__%(i)d__%(j)d__%(k)s;
    if (returnp > max_depthp) {
        %(check_pool)s;
    } else {
        val = map(%(len_rules)s);
    }
    goto *gen_%(t)s[val];
return__%(i)d__%(j)d__%(k)s:;
            ''' % {'i':i, 'j':j, 'k':self.k_to_s(k),
                   't':self.k_to_s(token), 'rnum':0, 'len_rules':len(trules), 'len_min_choices':len_min_choices, 'check_pool':check_pool})
            else:
                t = self.esc_char(token)
                res.append('''\
    *out_regionp++ = '%s';''' % t)
        return res, leaf
    
    def gen_alt_src_1rule(self, k):
        rule = self.grammar[k][0]
        ri = 0
        src, leaf = self.gen_rule_src(rule, k, ri)
        body = '\n'.join(src)
        result = []
        if leaf:
            return '''
gen_%(name)s_0: {
%(body)s
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}
        else:
             return '''
gen_%(name)s_0: {
    ++returnp;
    // single -- no switch
%(body)s
    --returnp;
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        ret = self.k_to_s(k)
        result = []
        if len(rules) == 1: return self.gen_alt_src_1rule(k)
        for ri, rule in enumerate(rules):
            src, leaf = self.gen_rule_src(rule, k, ri)
            body = '\n'.join(src)
            if leaf:
                result.append('''
gen_%(name)s_%(rnum)d: {
%(body)s
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
            else:
                 result.append('''
gen_%(name)s_%(rnum)d: {
    ++returnp;
%(body)s
    --returnp;
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
        return '\n'.join(result)

    def fuzz_out_var_defs(self):
        return '''\
extern char* out_regionp;'''
    
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);'''
    
    def fuzz_stack_var_defs(self):
        return '''
extern void* stackp[];
'''

    def fuzz_entry(self):
        result = ['''
void gen_init__(void** max_depthp) {
    uint8_t val;
    void** returnp = stackp;
    *returnp =  &&return__init;
''']
        for k in self.grammar:
            l = []
            for ri,rule in enumerate(self.grammar[k]):
                l.append('&&gen_%(k)s_%(ri)d' % {'k':self.k_to_s(k), 'ri':ri})
            s = '''
    void** gen_%(k)s[] = {
%(body)s
    };''' % {'k': self.k_to_s(k), 'body': ',\n'.join(l)}
            result.append(s)
        result.append('''
    goto gen_start_0;''')
        result.append(self.fuzz_fn_defs())
        result.append("""
return__init:
    *out_regionp++ = '\\n';
    return;
return_abort:
    exit(10); 
}""")
        return '\n'.join(result)

    
    def main_stack_var_defs(self):
        return'''
int max_depth;
void** max_depthp;
void* stackp[INT_MAX];
'''
    def main_init_var_defs(self):
        return'''
void gen_init__(void** max_depthp);
'''

    def fn_main_loop_frag(self):
        return '''
    fs = fdopen(out_fd, "w");
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depthp);
        out_cursor = out_regionp - out_region_initp;
        fwrite(out_region_initp, sizeof(char), out_cursor, fs);
    }
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_sizep = 0;
    int out_fd;
    int seed, max_num;
%(input_frag)s
    max_depthp = stackp + max_depth;
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }


    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          # self.fuzz_fn_defs(),
                          self.fuzz_entry()])

In [215]:
main_src, fuzz_src = CFWriteDTFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_fwritedt_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_fwritedt_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [216]:
!wc -l testers/grammar_producer_fwritedt_fuzz.c

     628 testers/grammar_producer_fwritedt_fuzz.c


In [217]:
!cat testers/grammar_producer_fwritedt_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>


int max_depth;
void** max_depthp;
void* stackp[INT_MAX];


void gen_init__(void** max_depthp);

uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;

const uint64_t size = UINT_MAX; /*max size of a single input -- 4G*/
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;
FILE* fs;


uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, i

In [218]:
%cd testers
!cc -g -Ofast -o grammar_producer_fwritedt grammar_producer_fwritedt_main.c grammar_producer_fwritedt_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [219]:
# II
class CTesterFWriteDT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_fwritedt {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
# II
!./testers/grammar_producer_fwritedt 0 10 10 io.x

In [ ]:
!du -ksh io.x

In [ ]:
CTesterFWriteDT().run_test().show()

#### Context threaded VM

In [220]:
# II
class CFWriteCTFuzzer(CFWriteDTFuzzer):
    
    def fn_choice(self, val):
        return '''
    # [ random 
    # extract one byte from the random stream %%r14,
    movq (%%r14), %%rdi
    # advance the random cursor
    inc %%r14                                     # rand_region++
    movzbl %%dil, %%edi                           # X  --- (rdi:(edi:(di:(dil))))
    # then multiply with the choices we have

    xor %%rsi, %%rsi                              # avoid data dependencies
    movb $%(val)s, %%sil                          # = %(val)s       
    movzbl %%sil, %%edx
    imull %%edi, %%edx                            # m = (short) x * (short) N)
    sarl $8, %%edx                                # return (char)(m >> 8) ;
    # random ]
    # %%edx now contains the selected random value from %(val)d options''' % {'val':val}

    def cheap_strings(self, k):
        cheap_strings = self.pool_of_strings[k]
        results = ['''
    # --- cheap -- [''']
        results.append('''
%(choices)s
''' % {'choices':self.fn_choice(len(cheap_strings)), 'len_choices': len(cheap_strings)})
        # get the choices from vm, then call it, and return.
        
        results.append('''
    # now we have the right print quad in %%edx. Load the right address and call it.
    leaq _%(key)s_prints(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ret
    ''' % {'key': self.k_to_s(k)})
        results.append('''
    # --- cheap -- ]''')
        return '\n'.join(results)
    
    def output_char(self, c):
        if len(c) != 1:
            assert c[0] == '\\'
            c = c[-1]
        return '''
   movb $%(ichar)d, (%%r13)                     # '%(char)s'
   inc %%r13                                    # out_region++   : increment a byte (r13++)
   ''' % {'char':self.esc(c), 'ichar':ord(c)}

    def gen_rule_src(self, rule, k, j):
        # in each rule, there are a number of tokens.
        # iter each token in turn, choose the right rule and call.
        result = []
        for token in rule:
            if token not in self.grammar:
                result.append(self.output_char(token))
                continue
            else:
                # how many choices do we have?
                rules = self.grammar[token]
                result.append('''
    # start the choice machine.
    # length of rules = %(len_rules)d
%(choices)s
    # --- switch ---
    ''' % {'choices': self.fn_choice(len(rules)), 'len_rules':len(rules)})
                result.append('''
    # now we have the right choice in %%edx. Load the right address and call it.
    leaq _%(key)s_choices(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ''' % {'key': self.k_to_s(token)})
        return '\n'.join(result)

    def gen_alt_src(self, k):
        result = []
        for ruleid, rule in enumerate(self.grammar[k]):
            # produce a skeletal subroutine structure.
            result.append('''
gen_%(key)s_%(ruleid)s:
    # check if the max depth is breached.
    cmpq %%rsp, %%r8                             # returnp(rbp) <> max_depth(r8) ?
    jle _%(key)s_%(ruleid)s_fi                       # returnp <= max_depth
    
%(return_cheap_string)s
_%(key)s_%(ruleid)s_fi:
''' % {'return_cheap_string': self.cheap_strings(k),
       'key':self.k_to_s(k),
       'ruleid':ruleid,
       'last_label':self.last_label})
            self.last_label += 1
            result.append(self.gen_rule_src(rule, k, ruleid))
            # we were called. So simply return.
            result.append('''
    ret
            ''')
        return '\n'.join(result)
 
    def fn_fuzz_decs(self):
        result = ['''
  .section  __DATA,__data

# Virtual Machine OPS.
        ''']
        for k in self.grammar:
            result.append('''
    .globl  _%(key)s_choices
    .p2align 4
_%(key)s_choices:''' % {'key':self.k_to_s(k)})
            for i, rule in enumerate(self.grammar[k]):
                result.append('''\
    .quad gen_%s_%d''' % (self.k_to_s(k), i))
                
        for k in self.pool_of_strings:
            result.append('''
    .globl  _%(key)s_prints
    .p2align 4
_%(key)s_prints:''' % {'key':self.k_to_s(k)})
            for string in self.pool_of_strings[k]:
                result.append('''\
    .quad %s''' % (self.all_prints[string]))
                
                
        result.append('''
# End Virtual Machine OPS.''')
        return '\n'.join(result)

    def gen_cheap(self, grammar):
        all_strings = set()
        for k in grammar:
            all_strings |= set(self.pool_of_strings[k])
        all_strings = list(all_strings)
        all_strings.sort(key=lambda item: (-len(item), item))
        all_prints_hash = {}
        result = ['''
.text
        ''']
        for i, s_ in enumerate(all_strings):
            s = s_
            result.append('''\
print_%(name)d: # "%(value)s"''' % {'name': i, 'value': self.esc(s)})
            for j in s:
                result.append('''\
    movb $%(ichar)s, (%%r13)            # '%(char)s'
    inc %%r13''' % {'ichar':ord(j), 'char':self.esc(j)})
            result.append('''\
    ret''')
            all_prints_hash[s_] = 'print_%d' % i
        return ('\n'.join(result), all_prints_hash)
 
    def fuzz_entry(self):
        result = ["""
#include "ctfwrite_vm_ops.s"
.macro pushaq
    push %%rsp
    push %%rbp
    push %%r8
    push %%r9
    push %%r10
    push %%r11
    push %%r12
    push %%r13
    push %%r14
    push %%r15
.endm


.macro popaq
    pop %%r15
    pop %%r14
    pop %%r13
    pop %%r12
    pop %%r11
    pop %%r10
    pop %%r9
    pop %%r8
    pop %%rbp
    pop %%rsp
.endm

.global %(os)sgen_init__
.global return__init
.text
%(os)sgen_init__:
    # 1 rdi = max_depth
    # 2 rsi = returnp
    # 3 rdx = &out_region
    # 4 rcx = &rand_region
    pushaq

    leal 0(,%%rdi,8), %%eax
    movq %%rsp, %%r8
    subq %%rax, %%r8

    movq %%rdx, %%r11                              # &out_region
    movq %%rcx, %%r12                              # &rand_region
    movq (%%r11),%%r13                             # out_region
    movq (%%r12),%%r14                             # rand_region

    # general regs
    # rax, rcx, rdx, rbx, rsi,rdi
    # rbp, r8-r15
    
    call gen_start_0
    movq %%r13, (%%r11)                            # *(&out_region) <-
    movq %%r14, (%%r12)                            # *(&rand_region) <-
    popaq
    movq  $0, %%rax
    ret   
""" % {'os': '_' if sys.platform == 'darwin' else ''}]
        result.append(self.fuzz_fn_defs())
        return ''.join(result)

    def main_init_var_defs(self):
        return'''
void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
'''

    def fn_main_loop_frag(self):
        return '''
    fs = fdopen(out_fd, "w");
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depth32, stackp, &out_regionp, &rand_regionp);
        *out_regionp++ = '\\n';
        out_cursor = out_regionp - out_region_initp;
        fwrite(out_region_initp, sizeof(char), out_cursor, fs);
    }
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    int out_fd;
    uint32_t max_depth32;
    int seed, max_num;
%(input_frag)s
    max_depth32 = max_depth;
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }
    
    def fuzz_src(self, key='<start>'):
        self.last_label = 0
        self.cheap, self.all_prints = self.gen_cheap(self.grammar)
        ext_strings = '\n'.join([self.fn_fuzz_decs(), self.cheap])
        return ext_strings, self.gen_main_src(), self.gen_fuzz_src()
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_entry()])

In [221]:
vm_ops, main_src, fuzz_src = CFWriteCTFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_ctfwrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_ctfwrite_fuzz.s', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/ctfwrite_vm_ops.s', 'w+') as f:
    print(vm_ops, file=f)

In [222]:
!nl -ba testers/ctfwrite_vm_ops.s

     1	
     2	  .section  __DATA,__data
     3	
     4	# Virtual Machine OPS.
     5	        
     6	
     7	    .globl  _start_choices
     8	    .p2align 4
     9	_start_choices:
    10	    .quad gen_start_0
    11	
    12	    .globl  _expr_choices
    13	    .p2align 4
    14	_expr_choices:
    15	    .quad gen_expr_0
    16	    .quad gen_expr_1
    17	    .quad gen_expr_2
    18	
    19	    .globl  _term_choices
    20	    .p2align 4
    21	_term_choices:
    22	    .quad gen_term_0
    23	    .quad gen_term_1
    24	    .quad gen_term_2
    25	
    26	    .globl  _factor_choices
    27	    .p2align 4
    28	_factor_choices:
    29	    .quad gen_factor_0
    30	    .quad gen_factor_1
    31	    .quad gen_factor_2
    32	    .quad gen_factor_3
    33	    .quad gen_factor_4
    34	
    35	    .globl  _integer_choices
    36	    .p2align 4
    37	_integer_choices:
    38	    .quad gen_integer_0
    39	    .quad gen_integer_1
    40	
    41	    .globl  _digit_choices
    42	    .p2ali

In [223]:
!wc -l testers/grammar_producer_ctfwrite_fuzz.s

    1634 testers/grammar_producer_ctfwrite_fuzz.s


In [224]:
!cat testers/grammar_producer_ctfwrite_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>


int max_depth;
void** max_depthp;
void* stackp[INT_MAX];


void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);

uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;

const uint64_t size = UINT_MAX; /*max size of a single input -- 4G*/
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;
FILE* fs;


uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to

In [225]:
%cd testers
!cc -g -Ofast -o grammar_producer_ctfwrite grammar_producer_ctfwrite_main.c grammar_producer_ctfwrite_fuzz.s
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
./ctfwrite_vm_ops.s:544:16: error: unknown token in expression
    movb $48, (%r13) # '0'
               ^
./ctfwrite_vm_ops.s:544:15: error: invalid operand
    movb $48, (%r13) # '0'
              ^
./ctfwrite_vm_ops.s:545:9: error: unknown token in expression
    inc %r13
        ^
./ctfwrite_vm_ops.s:545:9: error: invalid operand
    inc %r13
        ^
./ctfwrite_vm_ops.s:546:16: error: unknown token in expression
    movb $46, (%r13) # '.'
               ^
./ctfwrite_vm_ops.s:546:15: error: invalid operand
    movb $46, (%r13) # '.'
              ^
./ctfwrite_vm_ops.s:547:9: error: unknown token in expression
    inc %r13
        ^
./ctfwrite_vm_ops.s:547:9: error: invalid operand
    inc %r13
        ^
./ctfwrite_vm_ops.s:548:16: error: unknown token in expression
    movb $48, (%r13) # '0'
               ^
./ctfwrite_vm_ops.s:548:15: error: invalid operand
    movb $48, (%r13) # '0'
              ^
./ctfwrite_vm_ops.s:549:9: error: un

In [ ]:
# II
!./testers/grammar_producer_ctfwrite 0 10 10 io.x

In [ ]:
!du -ksh io.x

In [226]:
class CTesterFWriteCT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_ctfwrite {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterFWriteCT().run_test().show()

### No Output

#### Direct threaded VM

In [227]:
class DTNoWriteFuzzer(CNoWriteFuzzer):
    def fn_out_def(self): return ''
    def fn_main_out_frag(self): return ''
    def gen_rule_src(self, rule, k, j):
        res = []
        leaf = True
        for i, token in enumerate(rule):
            if token in self.grammar:
                leaf = False
                trules = self.grammar[token] # ordered by cost
                len_min_choices = len(self.c_grammar[token])
                assert len(trules) < 256
                cheap_strings = self.pool_of_strings[token]
                if len(cheap_strings) < 256: # we only have 255 random choices
                    check_pool = '''
        val = map(%(len_cheap_strings)s);
        const char* str = pool_%(k)s[val];
        const int str_l = pool_l_%(k)s[val];
        for (int i = 0; i < str_l; i++) {
            *out_regionp++ = str[i];
        }
        --returnp;
        goto **returnp; 
            ''' % { 'len_cheap_strings': len(cheap_strings), 'k': self.k_to_s(token)}
                else:
                    check_pool = '''
        val = map(%(len_min_choices)s);
                ''' % {'len_min_choices':len_min_choices}
                res.append('''\
    *returnp = &&return__%(i)d__%(j)d__%(k)s;
    if (returnp > max_depthp) {
        %(check_pool)s;
    } else {
        val = map(%(len_rules)s);
    }
    goto *gen_%(t)s[val];
return__%(i)d__%(j)d__%(k)s:;
            ''' % {'i':i, 'j':j, 'k':self.k_to_s(k),
                   't':self.k_to_s(token), 'rnum':0, 'len_rules':len(trules), 'len_min_choices':len_min_choices, 'check_pool':check_pool})
            else:
                t = self.esc_char(token)
                res.append('''\
    *out_regionp++ = '%s';''' % t)
        return res, leaf
    
    def gen_alt_src_1rule(self, k):
        rule = self.grammar[k][0]
        ri = 0
        src, leaf = self.gen_rule_src(rule, k, ri)
        body = '\n'.join(src)
        result = []
        if leaf:
            return '''
gen_%(name)s_0: {
%(body)s
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}
        else:
             return '''
gen_%(name)s_0: {
    ++returnp;
    // single -- no switch
%(body)s
    --returnp;
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        ret = self.k_to_s(k)
        result = []
        if len(rules) == 1: return self.gen_alt_src_1rule(k)
        for ri, rule in enumerate(rules):
            src, leaf = self.gen_rule_src(rule, k, ri)
            body = '\n'.join(src)
            if leaf:
                result.append('''
gen_%(name)s_%(rnum)d: {
%(body)s
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
            else:
                 result.append('''
gen_%(name)s_%(rnum)d: {
    ++returnp;
%(body)s
    --returnp;
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
        return '\n'.join(result)

    def fuzz_out_var_defs(self):
        return '''\
extern char* out_regionp;'''
    
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);'''
    
    def fuzz_stack_var_defs(self):
        return '''
extern void* stackp[];
'''

    def fuzz_entry(self):
        result = ['''
void gen_init__(void** max_depthp) {
    uint8_t val;
    void** returnp = stackp;
    *returnp =  &&return__init;
''']
        for k in self.grammar:
            l = []
            for ri,rule in enumerate(self.grammar[k]):
                l.append('&&gen_%(k)s_%(ri)d' % {'k':self.k_to_s(k), 'ri':ri})
            s = '''
    void** gen_%(k)s[] = {
%(body)s
    };''' % {'k': self.k_to_s(k), 'body': ',\n'.join(l)}
            result.append(s)
        result.append('''
    goto gen_start_0;''')
        result.append(self.fuzz_fn_defs())
        result.append("""
return__init:
    *out_regionp++ = '\\n';
    return;
return_abort:
    exit(10); 
}""")
        return '\n'.join(result)
    
    def main_stack_var_defs(self):
        return'''
int max_depth;
void** max_depthp;
void* stackp[INT_MAX];
'''
    def main_init_var_defs(self):
        return'''
void gen_init__(void** max_depthp);
'''

    def fn_main_loop_frag(self):
        return '''
    uint64_t out_size = 0;
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depthp);
        out_cursor = out_regionp - out_region_initp;
        out_size += out_cursor;
    }
    printf("%lld\\n", out_size);
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    char* out_region_sizep = 0;
    int out_fd;
    int seed, max_num;
%(input_frag)s
    max_depthp = stackp + max_depth;
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }


    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          # self.fuzz_fn_defs(),
                          self.fuzz_entry()])

In [228]:
main_src, fuzz_src = DTNoWriteFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_dtnowrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_dtnowrite_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [229]:
!cat testers/grammar_producer_dtnowrite_main.c


#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>

#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>


int max_depth;
void** max_depthp;
void* stackp[INT_MAX];


void gen_init__(void** max_depthp);

uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;

const uint64_t size = UINT_MAX; // size of a single output item -- 4G
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;

uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, int k) {
  

In [230]:
%cd testers
!cc -g -Ofast -o grammar_producer_dtnowrite grammar_producer_dtnowrite_main.c grammar_producer_dtnowrite_fuzz.c
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
/Users/rahul.gopinath/Research/f1/F1


In [ ]:
!./testers/grammar_producer_dtnowrite  0 10 10

In [231]:
class CTesterNoWriteDT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_dtnowrite {seed} {self.max_num} {max_depth} > {fn}"
  
    def post_time(self):
        super().post_time()
        with open(self.file) as f:
            self.size = int(f.read())

In [ ]:
CTesterNoWriteDT().run_test().show()

#### Context threaded VM

In [232]:
class CTNoWriteFuzzer(DTNoWriteFuzzer):
    def fn_main_out_frag(self): return ''
    def fn_main_sync_frag(self): return ''
    
    def fn_choice(self, val):
        return '''
    # [ random 
    # extract one byte from the random stream %%r14,
    movq (%%r14), %%rdi
    # advance the random cursor
    inc %%r14                                     # rand_region++
    movzbl %%dil, %%edi                           # X  --- (rdi:(edi:(di:(dil))))
    # then multiply with the choices we have

    xor %%rsi, %%rsi                              # avoid data dependencies
    movb $%(val)s, %%sil                          # = %(val)s       
    movzbl %%sil, %%edx
    imull %%edi, %%edx                            # m = (short) x * (short) N)
    sarl $8, %%edx                                # return (char)(m >> 8) ;
    # random ]
    # %%edx now contains the selected random value from %(val)d options''' % {'val':val}

    def cheap_strings(self, k):
        cheap_strings = self.pool_of_strings[k]
        results = ['''
    # --- cheap -- [''']
        results.append('''
%(choices)s
''' % {'choices':self.fn_choice(len(cheap_strings)), 'len_choices': len(cheap_strings)})
        # get the choices from vm, then call it, and return.
        
        results.append('''
    # now we have the right print quad in %%edx. Load the right address and call it.
    leaq _%(key)s_prints(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ret
    ''' % {'key': self.k_to_s(k)})
        results.append('''
    # --- cheap -- ]''')
        return '\n'.join(results)
    
    def output_char(self, c):
        if len(c) != 1:
            assert c[0] == '\\'
            c = c[-1]
        return '''
   movb $%(ichar)d, (%%r13)                     # '%(char)s'
   inc %%r13                                    # out_region++   : increment a byte (r13++)
   ''' % {'char':self.esc(c), 'ichar':ord(c)}

    def gen_rule_src(self, rule, k, j):
        # in each rule, there are a number of tokens.
        # iter each token in turn, choose the right rule and call.
        result = []
        for token in rule:
            if token not in self.grammar:
                result.append(self.output_char(token))
                continue
            else:
                # how many choices do we have?
                rules = self.grammar[token]
                result.append('''
    # start the choice machine.
    # length of rules = %(len_rules)d
%(choices)s
    # --- switch ---
    ''' % {'choices': self.fn_choice(len(rules)), 'len_rules':len(rules)})
                result.append('''
    # now we have the right choice in %%edx. Load the right address and call it.
    leaq _%(key)s_choices(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ''' % {'key': self.k_to_s(token)})
        return '\n'.join(result)

    def gen_alt_src(self, k):
        result = []
        for ruleid, rule in enumerate(self.grammar[k]):
            # produce a skeletal subroutine structure.
            result.append('''
gen_%(key)s_%(ruleid)s:
    # check if the max depth is breached.
    cmpq %%rsp, %%r8                             # returnp(rbp) <> max_depth(r8) ?
    jle _%(key)s_%(ruleid)s_fi                       # returnp <= max_depth
    
%(return_cheap_string)s
_%(key)s_%(ruleid)s_fi:
''' % {'return_cheap_string': self.cheap_strings(k),
       'key':self.k_to_s(k),
       'ruleid':ruleid,
       'last_label':self.last_label})
            self.last_label += 1
            result.append(self.gen_rule_src(rule, k, ruleid))
            # we were called. So simply return.
            result.append('''
    ret
            ''')
        return '\n'.join(result)
 
    def fn_fuzz_decs(self):
        result = ['''
  .section  __DATA,__data

# Virtual Machine OPS.
        ''']
        for k in self.grammar:
            result.append('''
    .globl  _%(key)s_choices
    .p2align 4
_%(key)s_choices:''' % {'key':self.k_to_s(k)})
            for i, rule in enumerate(self.grammar[k]):
                result.append('''\
    .quad gen_%s_%d''' % (self.k_to_s(k), i))
                
        for k in self.pool_of_strings:
            result.append('''
    .globl  _%(key)s_prints
    .p2align 4
_%(key)s_prints:''' % {'key':self.k_to_s(k)})
            for string in self.pool_of_strings[k]:
                result.append('''\
    .quad %s''' % (self.all_prints[string]))
                
                
        result.append('''
# End Virtual Machine OPS.''')
        return '\n'.join(result)

    def gen_cheap(self, grammar):
        all_strings = set()
        for k in grammar:
            all_strings |= set(self.pool_of_strings[k])
        all_strings = list(all_strings)
        all_strings.sort(key=lambda item: (-len(item), item))
        all_prints_hash = {}
        result = ['''
.text
        ''']
        for i, s_ in enumerate(all_strings):
            s = s_
            result.append('''\
print_%(name)d: # "%(value)s"''' % {'name': i, 'value': self.esc(s)})
            for j in s:
                result.append('''\
    movb $%(ichar)s, (%%r13)            # '%(char)s'
    inc %%r13''' % {'ichar':ord(j), 'char':self.esc(j)})
            result.append('''\
    ret''')
            all_prints_hash[s_] = 'print_%d' % i
        return ('\n'.join(result), all_prints_hash)
 
    def fuzz_entry(self):
        result = ["""
#include "ctnowrite_vm_ops.s"
.macro pushaq
    push %%rsp
    push %%rbp
    push %%r8
    push %%r9
    push %%r10
    push %%r11
    push %%r12
    push %%r13
    push %%r14
    push %%r15
.endm


.macro popaq
    pop %%r15
    pop %%r14
    pop %%r13
    pop %%r12
    pop %%r11
    pop %%r10
    pop %%r9
    pop %%r8
    pop %%rbp
    pop %%rsp
.endm

.global %(os)sgen_init__
.global return__init
.text
%(os)sgen_init__:
    # 1 rdi = max_depth
    # 2 rsi = returnp
    # 3 rdx = &out_region
    # 4 rcx = &rand_region
    pushaq

    leal 0(,%%rdi,8), %%eax
    movq %%rsp, %%r8
    subq %%rax, %%r8

    movq %%rdx, %%r11                              # &out_region
    movq %%rcx, %%r12                              # &rand_region
    movq (%%r11),%%r13                             # out_region
    movq (%%r12),%%r14                             # rand_region

    # general regs
    # rax, rcx, rdx, rbx, rsi,rdi
    # rbp, r8-r15
    
    call gen_start_0
    movq %%r13, (%%r11)                            # *(&out_region) <-
    movq %%r14, (%%r12)                            # *(&rand_region) <-
    popaq
    movq  $0, %%rax
    ret   
""" % {'os': '_' if sys.platform == 'darwin' else ''}]
        result.append(self.fuzz_fn_defs())
        return ''.join(result)

    def main_init_var_defs(self):
        return'''
void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
'''

    def fn_main_loop_frag(self):
        return '''
    uint64_t out_size = 0;
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depth32, stackp, &out_regionp, &rand_regionp);
        *out_regionp++ = '\\n';
        out_cursor = out_regionp - out_region_initp;
        out_size += out_cursor;
    }
    printf("%lld\\n", out_size);
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd;
    uint32_t max_depth32;
    int seed, max_num;
%(input_frag)s
    max_depth32 = max_depth;
%(rand_frag)s
%(loop_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag()
       }
    
    def fuzz_src(self, key='<start>'):
        self.last_label = 0
        self.cheap, self.all_prints = self.gen_cheap(self.grammar)
        ext_strings = '\n'.join([self.fn_fuzz_decs(), self.cheap])
        return ext_strings, self.gen_main_src(), self.gen_fuzz_src()
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_entry()])

In [233]:
vm_ops, main_src, fuzz_src = CTNoWriteFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_ctnowrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_ctnowrite_fuzz.s', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/ctnowrite_vm_ops.s', 'w+') as f:
    print(vm_ops, file=f)

In [234]:
%cd testers
!cc -g -Ofast -o grammar_producer_ctnowrite grammar_producer_ctnowrite_main.c grammar_producer_ctnowrite_fuzz.s
%cd ..

/Users/rahul.gopinath/Research/f1/F1/testers
./ctnowrite_vm_ops.s:544:16: error: unknown token in expression
    movb $48, (%r13) # '0'
               ^
./ctnowrite_vm_ops.s:544:15: error: invalid operand
    movb $48, (%r13) # '0'
              ^
./ctnowrite_vm_ops.s:545:9: error: unknown token in expression
    inc %r13
        ^
./ctnowrite_vm_ops.s:545:9: error: invalid operand
    inc %r13
        ^
./ctnowrite_vm_ops.s:546:16: error: unknown token in expression
    movb $46, (%r13) # '.'
               ^
./ctnowrite_vm_ops.s:546:15: error: invalid operand
    movb $46, (%r13) # '.'
              ^
./ctnowrite_vm_ops.s:547:9: error: unknown token in expression
    inc %r13
        ^
./ctnowrite_vm_ops.s:547:9: error: invalid operand
    inc %r13
        ^
./ctnowrite_vm_ops.s:548:16: error: unknown token in expression
    movb $48, (%r13) # '0'
               ^
./ctnowrite_vm_ops.s:548:15: error: invalid operand
    movb $48, (%r13) # '0'
              ^
./ctnowrite_vm_ops.s:549:9

In [235]:
!./testers/grammar_producer_ctnowrite 0 10 10

zsh:1: no such file or directory: ./testers/grammar_producer_ctnowrite


In [236]:
class CTesterNoWriteCT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_ctnowrite {seed} {self.max_num} {max_depth} > {fn}"
 
    def post_time(self):
        super().post_time()
        with open(self.file) as f:
            self.size = int(f.read())

In [ ]:
CTesterNoWriteCT().run_test().show()

# Results

In [237]:
for k in TX:
    print(k)
    for depth in TX[k]:
        print(depth)
        if 'avgruntime' not in TX[k][depth]: continue
        print('\truntime =',TX[k][depth]['avgruntime'])
        print('\tsize = ',TX[k][depth]['avgsize'])
        print('\tthroughput =',TX[k][depth]['avgthroughput'])
    print()
    
END_TIME = datetime.now()

RandomTester
8
	runtime = 0.035
	size =  14.5
	throughput = 0.40450750612745096
16
	runtime = 0.033
	size =  22.5
	throughput = 0.6658380681818181

FuzzingbookTester
8
	runtime = 2.1304999999999996
	size =  19665
	throughput = 9.014290071546165
16
	runtime = 10.063
	size =  47111
	throughput = 4.574883775538085

PyLimitTester
8
	runtime = 0.169
	size =  51554.5
	throughput = 297.9064580251479
16
	runtime = 0.559
	size =  240401.5
	throughput = 420.00049857019866

PyPooledTester
8
	runtime = 0.075
	size =  57931.5
	throughput = 754.9933946083881
16
	runtime = 0.2095
	size =  249138.5
	throughput = 1163.5161670390303

PyCompiledTester
8
	runtime = 0.056
	size =  57874
	throughput = 1015.0678585737179
16
	runtime = 0.1105
	size =  249101.5
	throughput = 2201.620525888104

PyPETester
8
	runtime = 0.054
	size =  57860
	throughput = 1046.902136042024
16
	runtime = 0.107
	size =  249097.5
	throughput = 2274.653531495741

PySupercompiledTester
8
	runtime = 0.047
	size =  57541
	throughput = 12

In [ ]:
str(END_TIME - START_TIME)

In [ ]:
import json

In [ ]:
!mkdir -p results

In [ ]:
from datetime import datetime
curtime = datetime.now().isoformat()
name = 'results/tx-%s.json' % curtime
with open(name, 'w+') as f:
    print(json.dumps(TX), file=f)
print(name)

# Grammar Transformations

## Speed vs Code size Tradeoffs

A sliding scale of

* Completely String pools
* Compile to a state machine (CFG to Regular expression of fixed depth)
* Encode depth in function name (Remove depth comparisons)

### Expanding the use of string pools from just closing to before max_depth is exhausted

 We are not stuck with a pool of strings only after exhaustion of max_depth. But we will have to account for differing probabilties of different strings if we want to achieve a distribution of strings as the original. Whether it is required to have the same distribution as the original is a different question (because the original is clearly non-optimial -- shallow paths have more chance of being explroed again).

## Inlining

For inlining, we simply iterate through each key in the grammar, and each rule corresponding to a single key. For each rule, we inline one level, which will give us a list of corresponding rules. This set of rules will replace the original rule for the key.

# Probabilistic Fuzzing

One of the problems with our dumb grammar fuzzer is that each alternative rule for a key expansion is given the same probability. Hence, given a JSON element that can be a boolean, number, object or an array, the boolean (true and false) values will occur very 5 elements. This is clearly non-optimal. Hence, we need to extend our fuzzer to include probabilities in the grammar definition.

Once we adopt probabilistic fuzzing, we can generate a probabilistic profile of the grammar rules by expanding the grammar to a given depth, and simply counting the number of complete items produced by each expansion. This can ensure that there is a high probability of exploring at least to that depth.

In [ ]:
from functools import reduce
import operator

def items_in_rule(grammar, rule, depth, max_depth):
    if depth > max_depth: return 1
    return reduce(operator.mul,
                  [items_in_key(grammar, key, depth+1, max_depth)
                   for key in  rule], 1)


def items_in_key(grammar, key, depth=0, max_depth=10):
    if key not in grammar: return 1
    return sum(items_in_rule(grammar, rule, depth, max_depth)
               for rule in grammar[key])

def explore_grammar(grammar, max_depth):
    new_g = {}
    for k in grammar:
        new_rules = []
        for rule in grammar[k]:
            items = items_in_rule(grammar, rule, depth=0, max_depth=max_depth)
            new_rule = (rule, items)
            new_rules.append(new_rule)
        new_g[k] = new_rules
    return new_g

def to_ranges(pgrammar):
    new_g = {}
    for k in pgrammar:
        last = 0
        elts = []
        for elt in pgrammar[k]:
            rule, count = elt
            frm = last
            last += count
            to = last
            new_elt = rule, (frm, to)
            elts.append(new_elt)
        new_g[k] = elts
    return new_g
p_grammar = explore_grammar(my_grammar, 1)

In [ ]:
p_grammar

In [ ]:
def get_included_rule(idx, rules_):
    for r,rng in rules_:
        if rng[0] <= idx and idx < rng[1]: return r
    assert False

def gen_key(grammar, key, depth=0, max_depth=10):
    if key not in grammar: return [key]
    if depth > max_depth: return [random.choice(pool_of_strings[key])]
    
    rules_ = grammar[key]
    max_val = max([j for rule, (i,j) in rules_])
    rule = get_included_rule(random.randrange(max_val), rules_)
    return gen_rule(grammar, rule, depth+1, max_depth)

def gen_rule(grammar, rule, depth, max_depth):
    return sum([gen_key(grammar, token, depth, max_depth) for token in rule], [])

def grammar_producer_p(grammar, key='<start>'):
    cp_grammar = to_ranges(grammar)
    return ''.join(gen_key(cp_grammar, key))

In [ ]:
pf = PooledFuzzer(my_grammar)
pool_of_strings = pf.pool_of_strings
for i in range(10):
    print(grammar_producer_p(p_grammar))

# Generating Large Inputs Fast